In [537]:
#| default_exp handlers.helcom

# HELCOM

> Data pipeline (handler) to convert HELCOM data ([source](https://helcom.fi/about-us)) to `NetCDF` format or `Open Refine` format.  

<!-- ## HELCOM MORS Environment database -->

[Helcom MORS data](https://helcom.fi/about-us) is provided as a Microsoft Access database. 
[`Mdbtools`](https://github.com/mdbtools/mdbtools) can be used to convert the tables of the Microsoft Access database to `.csv` files on Unix-like OS.

Example steps:
1. Download data (e.g. https://metadata.helcom.fi/geonetwork/srv/fin/catalog.search#/metadata/2fdd2d46-0329-40e3-bf96-cb08c7206a24). 
2. Install mdbtools via VScode Terminal 

    ```
    sudo apt-get -y install mdbtools
    ````

3. Install unzip via VScode Terminal 

    ```
    sudo apt-get -y install unzip
    ````

4. In VS code terminal, navigate to the marisco data folder

    ```
    cd /home/marisco/downloads/marisco/_data/accdb/mors_19840101_20211231
    ```

5. Unzip MORS_ENVIRONMENT.zip 

    ```
    unzip MORS_ENVIRONMENT.zip 
    ```

6. Run preprocess.sh to generate the required data files

    ```
    ./preprocess.sh MORS_ENVIRONMENT.zip
    ````
7. Conetens of 'preprocess.sh' script.
    ```
    #!/bin/bash

    # Example of use: ./preprocess.sh MORS_ENVIRONMENT.zip
    unzip $1
    dbname=$(ls *.accdb)
    mkdir csv
    for table in $(mdb-tables -1 "$dbname"); do
        echo "Export table $table"
        mdb-export "$dbname" "$table" > "csv/$table.csv"
    done
    ```

***

## Packages import

In [538]:
%load_ext autoreload
%autoreload 2

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [539]:
#| export
import pandas as pd # Python package that provides fast, flexible, and expressive data structures.
import numpy as np
from tqdm import tqdm # Python Progress Bar Library
from functools import partial # Function which Return a new partial object which when called will behave like func called with the positional arguments args and keyword arguments keywords
import fastcore.all as fc # package that brings fastcore functionality, see https://fastcore.fast.ai/.
from pathlib import Path # This module offers classes representing filesystem paths
from dataclasses import asdict
from typing import List, Dict, Callable, Optional, Tuple

from marisco.utils import (has_valid_varname, match_worms, match_maris_lut, Match)
from marisco.callbacks import (Callback, Transformer, EncodeTimeCB, SanitizeLonLatCB)
from marisco.metadata import (GlobAttrsFeeder, BboxCB, DepthRangeCB, TimeRangeCB, ZoteroCB, KeyValuePairCB)
from marisco.configs import (base_path, nc_tpl_path, cfg, cache_path, cdl_cfg, Enums, lut_path,
                             species_lut_path, sediments_lut_path, bodyparts_lut_path, 
                             detection_limit_lut_path, filtered_lut_path, area_lut_path)
from marisco.serializers import NetCDFEncoder
from collections.abc import Callable
from math import modf
import warnings
from marisco.netcdf_to_csv import (LookupTimeFromEncodedTime, GetSampleTypeCB,
                                   LookupNuclideByIdCB, ConvertLonLatCB, LookupUnitByIdCB,
                                   LookupValueTypeByIdCB, LookupSpeciesByIdCB, 
                                   LookupBodypartByIdCB, LookupSedimentTypeByIdCB)                                  
from marisco.serializers import OpenRefineCsvEncoder

In [540]:
warnings.filterwarnings('ignore')

***

##  MARIS NetCDF 
When MARISCO is installed, it uses `cdl.toml` to create the `maris-template.nc`, which acts as a standardized template for MARIS NetCDF files. The `cdl.toml` is a configuration file listing all the variables allowed in the NetCDF4 files. The contents of the cdl.toml can be retrieved with the function `cdl_cfg()`.  

Retrieving the keys of the `cdl_cfg()`.

In [541]:
print (cdl_cfg()['vars'].keys())

dict_keys(['defaults', 'bio', 'sed', 'suffixes'])


Printing the contents of all keys

In [542]:
print (cdl_cfg()['vars']['defaults'].keys())
print (cdl_cfg()['vars']['bio'].keys())
print (cdl_cfg()['vars']['sed'].keys())
print (cdl_cfg()['vars']['suffixes'].keys())

dict_keys(['lon', 'lat', 'smp_depth', 'tot_depth', 'time', 'area'])
dict_keys(['bio_group', 'species', 'body_part'])
dict_keys(['sed_type'])
dict_keys(['uncertainty', 'detection_limit', 'volume', 'salinity', 'temperature', 'filtered', 'counting_method', 'sampling_method', 'preparation_method', 'unit'])


***

## MARIS Open Refine 

Currently, updates to the MARIS database are facilitated through a standardized CSV file using Open Refine. Description of the variables included in this CSV file are provided at [Maris](https://maris.iaea.org/help/1132).
 



***

## MARIS Open Refine CSV & MARIS NetCDF variable relationship. 

The table below lists the MARIS variables in both MARIS Open Refine and MARIS NetCDF formats. Each variable's presence in both formats for the seawater (``sea``), biota (``bio``), and sediment (``sed``) groups is indicated with a checkmark (``✓``).


<style>
  table {
    width: 100%;
    border-collapse: collapse
  }

  td,
  th {
    border: 1px solid #000;
    padding: 5px;
    text-align: center
  }

  th {
    background-color: #f2f2f2
  }

  .open-refine {
    background-color: #fff;
    color: black;
    text-align: center

  }

  .netcdf {
    background-color: #e6e6e6;
    color: black;
    text-align: center
  }
</style>
<table>
  <thead>
    <tr>
      <th class="open-refine">Open Refine Variables</th>
      <th class="open-refine">sea</th>
      <th class="open-refine">bio</th>
      <th class="open-refine">sed</th>
      <th class="netcdf">sea</th>
      <th class="netcdf">bio</th>
      <th class="netcdf">sed</th>
      <th class="netcdf">NetCDF Variables</th>
    </tr>
  </thead>
  <tbody>
    <tr>
      <td class="open-refine">Sample type</td>
      <td class="open-refine">✓</td>
      <td class="open-refine">✓</td>
      <td class="open-refine">✓</td>
      <td class="netcdf">✓</td>
      <td class="netcdf">✓</td>
      <td class="netcdf">✓</td>
      <td class="netcdf">*Included as netcdf.group*</td>
    </tr>
    <tr>
      <td class="open-refine">Latitude decimal</td>
      <td class="open-refine">✓</td>
      <td class="open-refine">✓</td>
      <td class="open-refine">✓</td>
      <td class="netcdf">✓</td>
      <td class="netcdf">✓</td>
      <td class="netcdf">✓</td>
      <td class="netcdf">lat</td>
    </tr>
    <tr>
      <td class="open-refine">Longitude decimal</td>
      <td class="open-refine">✓</td>
      <td class="open-refine">✓</td>
      <td class="open-refine">✓</td>
      <td class="netcdf">✓</td>
      <td class="netcdf">✓</td>
      <td class="netcdf">✓</td>
      <td class="netcdf">lon</td>
    </tr>
    <tr>
      <td class="open-refine">Sampling start date</td>
      <td class="open-refine">✓</td>
      <td class="open-refine">✓</td>
      <td class="open-refine">✓</td>
      <td class="netcdf">✓</td>
      <td class="netcdf">✓</td>
      <td class="netcdf">✓</td>
      <td class="netcdf">time</td>
    </tr>
    <tr>
      <td class="open-refine">Sampling start time</td>
      <td class="open-refine">✓</td>
      <td class="open-refine">✓</td>
      <td class="open-refine">✓</td>
      <td class="netcdf">✓</td>
      <td class="netcdf">✓</td>
      <td class="netcdf">✓</td>
      <td class="netcdf">time</td>
    </tr>
    <tr>
      <td class="open-refine">Sampling end date</td>
      <td class="open-refine">✓</td>
      <td class="open-refine">✓</td>
      <td class="open-refine">✓</td>
      <td class="netcdf"></td>
      <td class="netcdf"></td>
      <td class="netcdf"></td>
      <td class="netcdf"></td>
    </tr>
    <tr>
      <td class="open-refine">Sampling end time</td>
      <td class="open-refine">✓</td>
      <td class="open-refine">✓</td>
      <td class="open-refine">✓</td>
      <td class="netcdf"></td>
      <td class="netcdf"></td>
      <td class="netcdf"></td>
      <td class="netcdf"></td>
    </tr>
    <tr>
      <td class="open-refine">Nuclide</td>
      <td class="open-refine">✓</td>
      <td class="open-refine">✓</td>
      <td class="open-refine">✓</td>
      <td class="netcdf">✓</td>
      <td class="netcdf">✓</td>
      <td class="netcdf">✓</td>
      <td class="netcdf">nuclide</td>
    </tr>
    <tr>
      <td class="open-refine">Value type</td>
      <td class="open-refine">✓</td>
      <td class="open-refine">✓</td>
      <td class="open-refine">✓</td>
      <td class="netcdf">✓</td>
      <td class="netcdf">✓</td>
      <td class="netcdf">✓</td>
      <td class="netcdf">detection_limit</td>
    </tr>
    <tr>
      <td class="open-refine">Unit</td>
      <td class="open-refine">✓</td>
      <td class="open-refine">✓</td>
      <td class="open-refine">✓</td>
      <td class="netcdf">✓</td>
      <td class="netcdf">✓</td>
      <td class="netcdf">✓</td>
      <td class="netcdf">unit</td>
    </tr>
    <tr>
      <td class="open-refine">Activity or MDA</td>
      <td class="open-refine">✓</td>
      <td class="open-refine">✓</td>
      <td class="open-refine">✓</td>
      <td class="netcdf">✓</td>
      <td class="netcdf">✓</td>
      <td class="netcdf">✓</td>
      <td class="netcdf">value</td>
    </tr>
    <tr>
      <td class="open-refine">Uncertainty</td>
      <td class="open-refine">✓</td>
      <td class="open-refine">✓</td>
      <td class="open-refine">✓</td>
      <td class="netcdf">✓</td>
      <td class="netcdf">✓</td>
      <td class="netcdf">✓</td>
      <td class="netcdf">uncertainty</td>
    </tr>
    <tr>
      <td class="open-refine">Sampling depth</td>
      <td class="open-refine">✓</td>
      <td class="open-refine"></td>
      <td class="open-refine">✓</td>
      <td class="netcdf">✓</td>
      <td class="netcdf">✓</td>
      <td class="netcdf">✓</td>
      <td class="netcdf">smp_depth</td>
    </tr>
    <tr>
      <td class="open-refine">Top</td>
      <td class="open-refine"></td>
      <td class="open-refine"></td>
      <td class="open-refine">✓</td>
      <td class="netcdf"></td>
      <td class="netcdf"></td>
      <td class="netcdf"></td>
      <td class="netcdf"></td>
    </tr>
    <tr>
      <td class="open-refine">Bottom</td>
      <td class="open-refine"></td>
      <td class="open-refine"></td>
      <td class="open-refine">✓</td>
      <td class="netcdf"></td>
      <td class="netcdf"></td>
      <td class="netcdf"></td>
      <td class="netcdf"></td>
    </tr>
    <tr>
      <td class="open-refine">Species</td>
      <td class="open-refine"></td>
      <td class="open-refine">✓</td>
      <td class="open-refine"></td>
      <td class="netcdf"></td>
      <td class="netcdf">✓</td>
      <td class="netcdf"></td>
      <td class="netcdf">species</td>
    </tr>
    <tr>
      <td class="open-refine">Body part</td>
      <td class="open-refine"></td>
      <td class="open-refine">✓</td>
      <td class="open-refine"></td>
      <td class="netcdf"></td>
      <td class="netcdf">✓</td>
      <td class="netcdf"></td>
      <td class="netcdf">body_part</td>
    </tr>
    <tr>
      <td class="open-refine"></td>
      <td class="open-refine"></td>
      <td class="open-refine"></td>
      <td class="open-refine"></td>
      <td class="netcdf"></td>
      <td class="netcdf">✓</td>
      <td class="netcdf"></td>
      <td class="netcdf">bio_group</td>
    </tr>
    <tr>
      <td class="open-refine">Salinity</td>
      <td class="open-refine">✓</td>
      <td class="open-refine"></td>
      <td class="open-refine"></td>
      <td class="netcdf">✓</td>
      <td class="netcdf">✓</td>
      <td class="netcdf">✓</td>
      <td class="netcdf">salinity</td>
    </tr>
    <tr>
      <td class="open-refine">Temperature</td>
      <td class="open-refine">✓</td>
      <td class="open-refine"></td>
      <td class="open-refine"></td>
      <td class="netcdf">✓</td>
      <td class="netcdf">✓</td>
      <td class="netcdf">✓</td>
      <td class="netcdf">temperature</td>
    </tr>
    <tr>
      <td class="open-refine">Filtered</td>
      <td class="open-refine">✓</td>
      <td class="open-refine"></td>
      <td class="open-refine"></td>
      <td class="netcdf">✓</td>
      <td class="netcdf">✓</td>
      <td class="netcdf">✓</td>
      <td class="netcdf">filtered</td>
    </tr>
    <tr>
      <td class="open-refine">Mesh size</td>
      <td class="open-refine">✓</td>
      <td class="open-refine"></td>
      <td class="open-refine"></td>
      <td class="netcdf"></td>
      <td class="netcdf"></td>
      <td class="netcdf"></td>
      <td class="netcdf"></td>
    </tr>
    <tr>
      <td class="open-refine">Quality flag</td>
      <td class="open-refine">✓</td>
      <td class="open-refine">✓</td>
      <td class="open-refine">✓</td>
      <td class="netcdf"></td>
      <td class="netcdf"></td>
      <td class="netcdf"></td>
      <td class="netcdf"></td>
    </tr>
    <tr>
      <td class="open-refine">Sediment type</td>
      <td class="open-refine"></td>
      <td class="open-refine"></td>
      <td class="open-refine">✓</td>
      <td class="netcdf"></td>
      <td class="netcdf"></td>
      <td class="netcdf">✓</td>
      <td class="netcdf">sed_type</td>
    </tr>
    <tr>
      <td class="open-refine">Dry weight</td>
      <td class="open-refine"></td>
      <td class="open-refine">✓</td>
      <td class="open-refine">✓</td>
      <td class="netcdf"></td>
      <td class="netcdf"></td>
      <td class="netcdf"></td>
      <td class="netcdf"></td>
    </tr>
    <tr>
      <td class="open-refine">Wet weight</td>
      <td class="open-refine"></td>
      <td class="open-refine">✓</td>
      <td class="open-refine">✓</td>
      <td class="netcdf"></td>
      <td class="netcdf"></td>
      <td class="netcdf"></td>
      <td class="netcdf"></td>
    </tr>
    <tr>
      <td class="open-refine">Dry/wet ratio</td>
      <td class="open-refine"></td>
      <td class="open-refine">✓</td>
      <td class="open-refine">✓</td>
      <td class="netcdf"></td>
      <td class="netcdf"></td>
      <td class="netcdf"></td>
      <td class="netcdf"></td>
    </tr>
    <tr>
      <td class="open-refine">Station ID</td>
      <td class="open-refine">✓</td>
      <td class="open-refine">✓</td>
      <td class="open-refine">✓</td>
      <td class="netcdf"></td>
      <td class="netcdf"></td>
      <td class="netcdf"></td>
      <td class="netcdf"></td>
    </tr>
    <tr>
      <td class="open-refine">Sample ID</td>
      <td class="open-refine">✓</td>
      <td class="open-refine">✓</td>
      <td class="open-refine">✓</td>
      <td class="netcdf">✓</td>
      <td class="netcdf">✓</td>
      <td class="netcdf">✓</td>
      <td class="netcdf">data_provider_sample_id</td>
    </tr>
    <tr>
      <td class="open-refine">Total depth</td>
      <td class="open-refine">✓</td>
      <td class="open-refine"></td>
      <td class="open-refine"></td>
      <td class="netcdf">✓</td>
      <td class="netcdf">✓</td>
      <td class="netcdf">✓</td>
      <td class="netcdf">tot_depth</td>
    </tr>
    <tr>
      <td class="open-refine">Profile or transect ID</td>
      <td class="open-refine">✓</td>
      <td class="open-refine">✓</td>
      <td class="open-refine">✓</td>
      <td class="netcdf"></td>
      <td class="netcdf"></td>
      <td class="netcdf"></td>
      <td class="netcdf"></td>
    </tr>
    <tr>
      <td class="open-refine">Sampling method</td>
      <td class="open-refine">✓</td>
      <td class="open-refine">✓</td>
      <td class="open-refine">✓</td>
      <td class="netcdf">✓</td>
      <td class="netcdf">✓</td>
      <td class="netcdf">✓</td>
      <td class="netcdf">sampling_method</td>
    </tr>
    <tr>
      <td class="open-refine">Preparation method</td>
      <td class="open-refine">✓</td>
      <td class="open-refine">✓</td>
      <td class="open-refine">✓</td>
      <td class="netcdf">✓</td>
      <td class="netcdf">✓</td>
      <td class="netcdf">✓</td>
      <td class="netcdf">preparation_method</td>
    </tr>
    <tr>
      <td class="open-refine">Drying method</td>
      <td class="open-refine"></td>
      <td class="open-refine">✓</td>
      <td class="open-refine">✓</td>
      <td class="netcdf"></td>
      <td class="netcdf"></td>
      <td class="netcdf"></td>
      <td class="netcdf"></td>
    </tr>
    <tr>
      <td class="open-refine">Counting method</td>
      <td class="open-refine">✓</td>
      <td class="open-refine">✓</td>
      <td class="open-refine">✓</td>
      <td class="netcdf">✓</td>
      <td class="netcdf">✓</td>
      <td class="netcdf">✓</td>
      <td class="netcdf">counting_method</td>
    </tr>
    <tr>
      <td class="open-refine">Sample notes</td>
      <td class="open-refine">✓</td>
      <td class="open-refine">✓</td>
      <td class="open-refine">✓</td>
      <td class="netcdf">✓</td>
      <td class="netcdf">✓</td>
      <td class="netcdf">✓</td>
      <td class="netcdf">sample_notes<sup>*1</sup></td>
    </tr>
    <tr>
      <td class="open-refine">Measurement notes</td>
      <td class="open-refine">✓</td>
      <td class="open-refine">✓</td>
      <td class="open-refine">✓</td>
      <td class="netcdf">✓</td>
      <td class="netcdf">✓</td>
      <td class="netcdf">✓</td>
      <td class="netcdf">measurement_notes<sup>*1</sup></td>
    </tr>
  </tbody>
</table>

<sup>*1</sup> The MARIS NetCDF does not currently support strings of variable length (i.e. vlen string data type).

***

## Define variables

1. **fname_in** - is the path to the folder containing the HELCOM data in CSV format. The path can be defined as a relative path. 

2. **fname_out_nc** - is the path and filename for the NetCDF output.The path can be defined as a relative path. 

3. **Zotero key** - is used to retrieve attributes related to the dataset from [Zotero](https://www.zotero.org/). The MARIS datasets include a [library](https://maris.iaea.org/datasets) available on [Zotero](https://www.zotero.org/groups/2432820/maris/library). 


In [543]:
# | export
fname_in = '../../_data/accdb/mors/csv'
fname_out_nc = '../../_data/output/100-HELCOM-MORS-2024.nc'
fname_out_csv = '../../_data/output/100-HELCOM-MORS-2024.csv'
zotero_key ='26VMZZ2Q'
ref_id = 100

***

## Utils

In [544]:
#| export

def load_data(src_dir: str, smp_types: List[str] = ['SEA', 'SED', 'BIO']) -> Dict[str, pd.DataFrame]:
    """
    Load HELCOM data and return the data in a dictionary of dataframes with the dictionary key as the sample type.
    
    Args:
    src_dir (str): The directory where the source CSV files are located.
    smp_types (List[str]): A list of sample types to load. Defaults to ['SEA', 'SED', 'BIO'].
    
    Returns:
    Dict[str, pd.DataFrame]: A dictionary with sample types as keys and their corresponding dataframes as values.
    """
    dfs = {}
    lut_smp_type = {'SEA': 'seawater', 'SED': 'sediment', 'BIO': 'biota'}
    
    for smp_type in smp_types:
        fname_meas = smp_type + '02.csv'  # Measurement (i.e., radioactivity) information
        fname_smp = smp_type + '01.csv'  # Sample information
        
        df_meas = pd.read_csv(Path(src_dir) / fname_meas)
        df_smp = pd.read_csv(Path(src_dir) / fname_smp)
        
        df = pd.merge(df_meas, df_smp, on='KEY', how='left')
        dfs[lut_smp_type[smp_type]] = df
    
    return dfs


***

## Load data

`dfs` is a dictionary of dataframes created from the Helcom dataset located at the path `fname_in`. The data to be included in each dataframe is sorted by sample type. Each dictionary is defined with a key equal to the sample type. 

In [545]:
#|eval: false
dfs = load_data(fname_in)
print(dfs.keys())
print(f"Seawater cols: {dfs['seawater'].columns}")
print(f"Sediment cols: {dfs['sediment'].columns}")
print(f"Biota cols: {dfs['biota'].columns}")

dict_keys(['seawater', 'sediment', 'biota'])
Seawater cols: Index(['KEY', 'NUCLIDE', 'METHOD', '< VALUE_Bq/m³', 'VALUE_Bq/m³', 'ERROR%_m³',
       'DATE_OF_ENTRY_x', 'COUNTRY', 'LABORATORY', 'SEQUENCE', 'DATE', 'YEAR',
       'MONTH', 'DAY', 'STATION', 'LATITUDE (ddmmmm)', 'LATITUDE (dddddd)',
       'LONGITUDE (ddmmmm)', 'LONGITUDE (dddddd)', 'TDEPTH', 'SDEPTH', 'SALIN',
       'TTEMP', 'FILT', 'MORS_SUBBASIN', 'HELCOM_SUBBASIN', 'DATE_OF_ENTRY_y'],
      dtype='object')
Sediment cols: Index(['KEY', 'NUCLIDE', 'METHOD', '< VALUE_Bq/kg', 'VALUE_Bq/kg', 'ERROR%_kg',
       '< VALUE_Bq/m²', 'VALUE_Bq/m²', 'ERROR%_m²', 'DATE_OF_ENTRY_x',
       'COUNTRY', 'LABORATORY', 'SEQUENCE', 'DATE', 'YEAR', 'MONTH', 'DAY',
       'STATION', 'LATITUDE (ddmmmm)', 'LATITUDE (dddddd)',
       'LONGITUDE (ddmmmm)', 'LONGITUDE (dddddd)', 'DEVICE', 'TDEPTH',
       'UPPSLI', 'LOWSLI', 'AREA', 'SEDI', 'OXIC', 'DW%', 'LOI%',
       'MORS_SUBBASIN', 'HELCOM_SUBBASIN', 'SUM_LINK', 'DATE_OF_ENTRY_y'],
      dty

Show the structure of the `seawater` dataframe:

In [546]:
#|eval: false
dfs['seawater'].head()

,KEY,NUCLIDE,METHOD,< VALUE_Bq/m³,VALUE_Bq/m³,ERROR%_m³,DATE_OF_ENTRY_x,COUNTRY,LABORATORY,SEQUENCE,...,LONGITUDE (ddmmmm),LONGITUDE (dddddd),TDEPTH,SDEPTH,SALIN,TTEMP,FILT,MORS_SUBBASIN,HELCOM_SUBBASIN,DATE_OF_ENTRY_y
0,WKRIL2012003,CS137,NaN,NaN,5.3,32.0,08/20/14 00:00:00,90,KRIL,2012003,...,29.20,29.3333,NaN,0.0,NaN,NaN,NaN,11,11,08/20/14 00:00:00
1,WKRIL2012004,CS137,NaN,NaN,19.9,20.0,08/20/14 00:00:00,90,KRIL,2012004,...,29.20,29.3333,NaN,29.0,NaN,NaN,NaN,11,11,08/20/14 00:00:00
2,WKRIL2012005,CS137,NaN,NaN,25.5,20.0,08/20/14 00:00:00,90,KRIL,2012005,...,23.09,23.1500,NaN,0.0,NaN,NaN,NaN,11,3,08/20/14 00:00:00
3,WKRIL2012006,CS137,NaN,NaN,17.0,29.0,08/20/14 00:00:00,90,KRIL,2012006,...,27.59,27.9833,NaN,0.0,NaN,NaN,NaN,11,11,08/20/14 00:00:00
4,WKRIL2012007,CS137,NaN,NaN,22.2,18.0,08/20/14 00:00:00,90,KRIL,2012007,...,27.59,27.9833,NaN,39.0,NaN,NaN,NaN,11,11,08/20/14 00:00:00


Show the structure of the `biota` dataframe:

In [547]:
#|eval: false
dfs['biota'].head()

,KEY,NUCLIDE,METHOD,< VALUE_Bq/kg,VALUE_Bq/kg,BASIS,ERROR%,NUMBER,DATE_OF_ENTRY_x,COUNTRY,...,BIOTATYPE,TISSUE,NO,LENGTH,WEIGHT,DW%,LOI%,MORS_SUBBASIN,HELCOM_SUBBASIN,DATE_OF_ENTRY_y
0,BVTIG2012041,CS134,VTIG01,<,0.010140,W,NaN,NaN,02/27/14 00:00:00,6.0,...,F,5,16.0,45.7,948.0,18.453,92.9,2,16,02/27/14 00:00:00
1,BVTIG2012041,K40,VTIG01,,135.300000,W,3.57,NaN,02/27/14 00:00:00,6.0,...,F,5,16.0,45.7,948.0,18.453,92.9,2,16,02/27/14 00:00:00
2,BVTIG2012041,CO60,VTIG01,<,0.013980,W,NaN,NaN,02/27/14 00:00:00,6.0,...,F,5,16.0,45.7,948.0,18.453,92.9,2,16,02/27/14 00:00:00
3,BVTIG2012041,CS137,VTIG01,,4.338000,W,3.48,NaN,02/27/14 00:00:00,6.0,...,F,5,16.0,45.7,948.0,18.453,92.9,2,16,02/27/14 00:00:00
4,BVTIG2012040,CS134,VTIG01,<,0.009614,W,NaN,NaN,02/27/14 00:00:00,6.0,...,F,5,17.0,45.9,964.0,18.458,92.9,2,16,02/27/14 00:00:00


Show the structure of the `sediment` dataframe: 

In [548]:
#|eval: false
dfs['sediment'].head()

,KEY,NUCLIDE,METHOD,< VALUE_Bq/kg,VALUE_Bq/kg,ERROR%_kg,< VALUE_Bq/m²,VALUE_Bq/m²,ERROR%_m²,DATE_OF_ENTRY_x,...,LOWSLI,AREA,SEDI,OXIC,DW%,LOI%,MORS_SUBBASIN,HELCOM_SUBBASIN,SUM_LINK,DATE_OF_ENTRY_y
0,SKRIL2012048,RA226,NaN,NaN,35.0,26.0,NaN,NaN,NaN,08/20/14 00:00:00,...,20.0,0.006,NaN,NaN,NaN,NaN,11.0,11.0,NaN,08/20/14 00:00:00
1,SKRIL2012049,RA226,NaN,NaN,36.0,22.0,NaN,NaN,NaN,08/20/14 00:00:00,...,27.0,0.006,NaN,NaN,NaN,NaN,11.0,11.0,NaN,08/20/14 00:00:00
2,SKRIL2012050,RA226,NaN,NaN,38.0,24.0,NaN,NaN,NaN,08/20/14 00:00:00,...,2.0,0.006,NaN,NaN,NaN,NaN,11.0,11.0,NaN,08/20/14 00:00:00
3,SKRIL2012051,RA226,NaN,NaN,36.0,25.0,NaN,NaN,NaN,08/20/14 00:00:00,...,4.0,0.006,NaN,NaN,NaN,NaN,11.0,11.0,NaN,08/20/14 00:00:00
4,SKRIL2012052,RA226,NaN,NaN,30.0,23.0,NaN,NaN,NaN,08/20/14 00:00:00,...,6.0,0.006,NaN,NaN,NaN,NaN,11.0,11.0,NaN,08/20/14 00:00:00


***

## Data transformation pipeline for NetCDF.

### Data transformation pipeline utils

``CompareDfsAndTfm`` compares the original dataframes to the transformed dataframe. A dictionary of dataframes, ``tfm.dfs_dropped``, is created to include the data present in the original dataset but absent from the transformed data. ``tfm.compare_stats`` provides a quick overview of the number of rows in both the original dataframes and the transformed dataframe.

In [549]:
#| export
import pandas as pd
import numpy as np
from typing import List, Dict
from marisco.callbacks import Callback, Transformer

class CompareDfsAndTfm(Callback):
    "Create a dataframe of dropped data. Data included in the `dfs` not in the `tfm`."
    
    def __init__(self, dfs: Dict[str, pd.DataFrame]):
        fc.store_attr()
    
    def __call__(self, tfm: Transformer) -> None:
        self._initialize_tfm_attributes(tfm)
        for grp in tfm.dfs.keys():
            dropped_df = self._get_dropped_data(grp, tfm)
            tfm.dfs_dropped[grp] = dropped_df
            tfm.compare_stats[grp] = self._compute_stats(grp, tfm)

    def _initialize_tfm_attributes(self, tfm: Transformer) -> None:
        """Initialize attributes in `tfm`."""
        tfm.dfs_dropped = {}
        tfm.compare_stats = {}

    def _get_dropped_data(self, grp: str, tfm: Transformer) -> pd.DataFrame:
        """
        Get the data that is present in `dfs` but not in `tfm.dfs`.
        
        Args:
        grp (str): The group key.
        tfm (Transformer): The transformation object containing `dfs`.
        
        Returns:
        pd.DataFrame: Dataframe with dropped rows.
        """
        index_diff = self.dfs[grp].index.difference(tfm.dfs[grp].index)
        return self.dfs[grp].loc[index_diff]
    
    def _compute_stats(self, grp: str, tfm: Transformer) -> Dict[str, int]:
        """
        Compute comparison statistics between `dfs` and `tfm.dfs`.
        
        Args:
        grp (str): The group key.
        tfm (Transformer): The transformation object containing `dfs`.
        
        Returns:
        Dict[str, int]: Dictionary with comparison statistics.
        """
        return {
            'Number of rows in dfs': len(self.dfs[grp].index),
            'Number of rows in tfm.dfs': len(tfm.dfs[grp].index),
            'Number of dropped rows': len(tfm.dfs_dropped[grp].index),
            'Number of rows in tfm.dfs + Number of dropped rows': len(tfm.dfs[grp].index) + len(tfm.dfs_dropped[grp].index)
        }


### Normalize nuclide names

&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;*NetCDF4 format variable: ``nuclide``.*

&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;*Open Refine format variable: ``Nuclide``.*

#### Lower & strip nuclide names

Create a callback function, `LowerStripRdnNameCB`, that receives a dictionary of dataframes. For each dataframe in the dictionary, it converts the contents of the `Nuclides` column to lowercase and removes any leading or trailing whitespace.

In [550]:
#| export
class LowerStripRdnNameCB(Callback):
    """Convert nuclide names to lowercase and strip any trailing spaces."""

    def __call__(self, tfm):
        for key in tfm.dfs.keys():
            self._process_nuclide_column(tfm.dfs[key])

    def _process_nuclide_column(self, df):
        """Apply transformation to the 'NUCLIDE' column of the given DataFrame."""
        df['NUCLIDE'] = df['NUCLIDE'].apply(self._transform_nuclide)

    def _transform_nuclide(self, nuclide):
        """Convert nuclide name to lowercase and strip trailing spaces."""
        return nuclide.lower().strip()


Here we call a transformer, which applies the callback (e.g. `LowerStripRdnNameCB`) to the dictionary of dataframes, `dfs`. We then print the unique entries of the transformed `NUCLIDE` column for each dataframe included in the dictionary of dataframes, `dfs`.

In [551]:
#|eval: false
dfs = load_data(fname_in)
tfm = Transformer(dfs, cbs=[LowerStripRdnNameCB()])
print('seawater nuclides: ')
print(tfm()['seawater']['NUCLIDE'].unique())
print('biota nuclides: ')
print(tfm()['biota']['NUCLIDE'].unique())
print('sediment nuclides: ')
print(tfm()['sediment']['NUCLIDE'].unique())

seawater nuclides: 
['cs137' 'sr90' 'h3' 'cs134' 'pu238' 'pu239240' 'am241' 'cm242' 'cm244'
 'tc99' 'k40' 'ru103' 'sr89' 'sb125' 'nb95' 'ru106' 'zr95' 'ag110m'
 'cm243244' 'ba140' 'ce144' 'u234' 'u238' 'co60' 'pu239' 'pb210' 'po210'
 'np237' 'pu240' 'mn54']
biota nuclides: 
['cs134' 'k40' 'co60' 'cs137' 'sr90' 'ag108m' 'mn54' 'co58' 'ag110m'
 'zn65' 'sb125' 'pu239240' 'ru106' 'be7' 'ce144' 'pb210' 'po210' 'sb124'
 'sr89' 'zr95' 'te129m' 'ru103' 'nb95' 'ce141' 'la140' 'i131' 'ba140'
 'pu238' 'u235' 'bi214' 'pb214' 'pb212' 'tl208' 'ac228' 'ra223' 'eu155'
 'ra226' 'gd153' 'sn113' 'fe59' 'tc99' 'co57' 'sn117m' 'eu152' 'sc46'
 'rb86' 'ra224' 'th232' 'cs134137' 'am241' 'ra228' 'th228' 'k-40' 'cs138'
 'cs139' 'cs140' 'cs141' 'cs142' 'cs143' 'cs144' 'cs145' 'cs146']
sediment nuclides: 
['ra226' 'cs137' 'ra228' 'k40' 'sr90' 'cs134137' 'cs134' 'pu239240'
 'pu238' 'co60' 'ru103' 'ru106' 'sb125' 'ag110m' 'ce144' 'am241' 'be7'
 'th228' 'pb210' 'co58' 'mn54' 'zr95' 'ba140' 'po210' 'ra224' 'nb95'
 'p

#### Remap HELCOM nuclide names to MARIS nuclide names

The `maris-template.nc` file, which  is created from the `cdl.toml` on installation of the Marisco package, provides details of the nuclides permitted in the  MARIS NetCDF file. Here we define a function  `get_unique_nuclides()` which creates a list of the unique nuclides from each dataframe in the dictionary of dataframes `dfs`. The function `has_valid_varname` checks that each nuclide in this list is included in the `maris-template.nc` (i.e. the `cdl.toml`). `has_valid_varname` returns all variables in the list that are not in the `maris-template.nc` or returns `True`. 
 

In [552]:
#| export
def get_unique_nuclides(dfs: Dict[str, pd.DataFrame]) -> List[str]:
    """
    Get a list of unique radionuclide types measured across samples.

    Args:
        dfs (Dict[str, pd.DataFrame]): A dictionary where keys are sample names and values are DataFrames.

    Returns:
        List[str]: A list of unique radionuclide types.
    """
    # Collect unique nuclide names from all DataFrames
    nuclides = set()
    for df in dfs.values():
        nuclides.update(df['NUCLIDE'].unique())

    return list(nuclides)

In [553]:
#|eval: false
# Check if these variable names are consistent with MARIS CDL
has_valid_varname(get_unique_nuclides(tfm.dfs), nc_tpl_path())

"cs143" variable name not found in MARIS CDL
"cs139" variable name not found in MARIS CDL
"cs144" variable name not found in MARIS CDL
"cs142" variable name not found in MARIS CDL
"k-40" variable name not found in MARIS CDL
"cs146" variable name not found in MARIS CDL
"cs145" variable name not found in MARIS CDL
"cs134137" variable name not found in MARIS CDL
"pu239240" variable name not found in MARIS CDL
"cs141" variable name not found in MARIS CDL
"cm243244" variable name not found in MARIS CDL
"cs138" variable name not found in MARIS CDL
"pu238240" variable name not found in MARIS CDL
"cs140" variable name not found in MARIS CDL


False

Many nuclide names are not listed in the `maris-template.nc`. Here we create a look up table, `varnames_lut_updates`, which will be used to correct the nuclide names in the dictionary of dataframes (i.e. dfs) that are not compatible with the `maris-template.nc`.

In [554]:
#| export
varnames_lut_updates = {
    'k-40': 'k40',
    'cm243244': 'cm243_244_tot',
    'cs134137': 'cs134_137_tot',
    'pu239240': 'pu239_240_tot',
    'pu238240': 'pu238_240_tot',
    'cs138': 'cs137',
    'cs139': 'cs137',
    'cs140': 'cs137',
    'cs141': 'cs137',
    'cs142': 'cs137',
    'cs143': 'cs137',
    'cs144': 'cs137',
    'cs145': 'cs137',
    'cs146': 'cs137'}

Function `get_varnames_lut` returns a dictionary of nuclide names. This dictionary includes the `NUCLIDE` names from the dataframes in dfs, along with corrections specified in `varnames_lut_updates`.

In [555]:
#| export
def get_varnames_lut(
    dfs: Dict[str, pd.DataFrame], 
    lut: Dict[str, str] = varnames_lut_updates
) -> Dict[str, str]:
    """
    Generate a lookup table for radionuclide names, updating with provided mappings.

    Args:
        dfs (Dict[str, pd.DataFrame]): A dictionary where keys are sample names and values are DataFrames.
        lut (Dict[str, str], optional): A dictionary with additional mappings to update the lookup table.

    Returns:
        Dict[str, str]: A dictionary mapping radionuclide names to their corresponding names.
    """
    # Generate a base lookup table from unique nuclide names
    unique_nuclides = get_unique_nuclides(dfs)
    base_lut = {name: name for name in unique_nuclides}

    # Update the base lookup table with additional mappings
    base_lut.update(lut)
    
    return base_lut

Create a callback that remaps the nuclide names in the dataframes within dfs to the updated names in `varnames_lut_updates`.

In [556]:
# | export
class RemapRdnNameCB(Callback):
    """Remap radionuclide names to MARIS radionuclide names."""

    def __init__(self, fn_lut: Callable[[Dict[str, pd.DataFrame]], Dict[str, str]] =  partial(get_varnames_lut, lut=varnames_lut_updates)):
        """
        Initialize the RemapRdnNameCB with a function to generate the lookup table.

        Args:
            fn_lut (Callable, optional): A function that takes a dictionary of DataFrames and returns a lookup table.
        """
        fc.store_attr()

    def __call__(self, tfm: 'Transformer'):
        """
        Apply the lookup table to remap radionuclide names in DataFrames.

        Args:
            tfm (Transformer): The transformer object containing DataFrames.
        """
        lut = self.fn_lut(tfm.dfs)
        for grp in tfm.dfs:
            self._remap_nuclide_names(tfm.dfs[grp], lut)
    
    def _remap_nuclide_names(self, df: pd.DataFrame, lut: Dict[str, str]):
        """
        Remap radionuclide names in the 'NUCLIDE' column of the DataFrame.

        Args:
            df (pd.DataFrame): DataFrame containing the 'NUCLIDE' column.
            lut (Dict[str, str]): Lookup table for remapping radionuclide names.
        """
        if 'NUCLIDE' in df.columns:
            df['NUCLIDE'].replace(lut, inplace=True)
        else:
            raise ValueError("DataFrame must contain a 'NUCLIDE' column.")


Apply the transformer for callbacks `LowerStripRdnNameCB` and `RemapRdnNameCB`. Then, print the unique nuclides for each dataframe in the dictionary dfs.

In [557]:
#|eval: false
dfs = load_data(fname_in)
tfm = Transformer(dfs, cbs=[LowerStripRdnNameCB(),
                            RemapRdnNameCB(),
                            #CompareDfsAndTfm(dfs)
                            ])
tfm()

#print(pd.DataFrame.from_dict(tfm.compare_stats) , '\n')
print('seawater nuclides: ')
print(tfm.dfs['seawater']['NUCLIDE'].unique())
print('biota nuclides: ')
print(tfm.dfs['biota']['NUCLIDE'].unique())
print('sediment nuclides: ')
print(tfm.dfs['sediment']['NUCLIDE'].unique())

seawater nuclides: 
['cs137' 'sr90' 'h3' 'cs134' 'pu238' 'pu239_240_tot' 'am241' 'cm242'
 'cm244' 'tc99' 'k40' 'ru103' 'sr89' 'sb125' 'nb95' 'ru106' 'zr95'
 'ag110m' 'cm243_244_tot' 'ba140' 'ce144' 'u234' 'u238' 'co60' 'pu239'
 'pb210' 'po210' 'np237' 'pu240' 'mn54']
biota nuclides: 
['cs134' 'k40' 'co60' 'cs137' 'sr90' 'ag108m' 'mn54' 'co58' 'ag110m'
 'zn65' 'sb125' 'pu239_240_tot' 'ru106' 'be7' 'ce144' 'pb210' 'po210'
 'sb124' 'sr89' 'zr95' 'te129m' 'ru103' 'nb95' 'ce141' 'la140' 'i131'
 'ba140' 'pu238' 'u235' 'bi214' 'pb214' 'pb212' 'tl208' 'ac228' 'ra223'
 'eu155' 'ra226' 'gd153' 'sn113' 'fe59' 'tc99' 'co57' 'sn117m' 'eu152'
 'sc46' 'rb86' 'ra224' 'th232' 'cs134_137_tot' 'am241' 'ra228' 'th228']
sediment nuclides: 
['ra226' 'cs137' 'ra228' 'k40' 'sr90' 'cs134_137_tot' 'cs134'
 'pu239_240_tot' 'pu238' 'co60' 'ru103' 'ru106' 'sb125' 'ag110m' 'ce144'
 'am241' 'be7' 'th228' 'pb210' 'co58' 'mn54' 'zr95' 'ba140' 'po210'
 'ra224' 'nb95' 'pu238_240_tot' 'pu241' 'pu239' 'eu155' 'ir192' 'th2

After apply correction to the nuclide names check that all nuclide in the dictionary of dataframees are valid. Returns `True` if all are valid.

In [558]:
#|eval: false
has_valid_varname(get_unique_nuclides(tfm.dfs), nc_tpl_path())

True

***

### Parse time

&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;*NetCDF4 format variable: `time`.*

&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;*Open Refine format variables: `Sampling start date` and `Sampling start time`.*

Create a callback that remaps the time format in the dictionary of dataframes (i.e. `%m/%d/%y %H:%M:%S`):

In [559]:
#| export
class ParseTimeCB(Callback):
    def __init__(self):
        fc.store_attr()
            
        
    def __call__(self, tfm):
        for grp in tfm.dfs.keys():
            df = tfm.dfs[grp]
            self._process_dates(df)

    def _process_dates(self, df: pd.DataFrame):
        """
        Process and correct date and time information in the DataFrame.

        Args:
            df (pd.DataFrame): DataFrame containing the 'DATE', 'YEAR', 'MONTH', and 'DAY' columns.
        """
        # get 'time' from 'DATE' column
        df['time'] = pd.to_datetime(df['DATE'], format='%m/%d/%y %H:%M:%S')
        # if 'DATE' column is nan, get 'time' from 'YEAR','MONTH' and 'DAY' column. 
        # if 'DAY' or 'MONTH' is 0 then set it to 1. 
        df.loc[df["DAY"] == 0, "DAY"] = 1
        df.loc[df["MONTH"] == 0, "MONTH"] = 1
        
        # if 'DAY' and 'MONTH' is nan but YEAR is not nan then set 'DAY' and 'MONTH' both to 1. 
        condition = (df["DAY"].isna()) & (df["MONTH"].isna()) & (df["YEAR"].notna())
        df.loc[condition, "DAY"] = 1
        df.loc[condition, "MONTH"] = 1
        
        condition = df['DATE'].isna() # if 'DATE' is nan. 
        df['time']  = np.where(condition,
                                            # 'coerce', then invalid parsing will be set as NaT. NaT will result if the number of days are not valid for the month.
                                        pd.to_datetime(df[['YEAR', 'MONTH', 'DAY']], format='%y%m%d', errors='coerce'),  
                                        pd.to_datetime(df['DATE'], format='%m/%d/%y %H:%M:%S'))

Apply the transformer for callbacks `LowerStripRdnNameCB`, `RemapRdnNameCB` and `ParseTimeCB`. Then, print the `time` data for `seawater`.

In [560]:
#|eval: false
dfs = load_data(fname_in)
tfm = Transformer(dfs, cbs=[LowerStripRdnNameCB(),
                            RemapRdnNameCB(),
                            ParseTimeCB(),
                            CompareDfsAndTfm(dfs)
                            ])
tfm()
print(pd.DataFrame.from_dict(tfm.compare_stats) , '\n')
#print(tfm.dfs['seawater']['time'])

                                                    seawater  sediment  biota
Number of rows in dfs                                  20318     37347  14893
Number of rows in tfm.dfs                              20318     37347  14893
Number of dropped rows                                     0         0      0
Number of rows in tfm.dfs + Number of dropped rows     20318     37347  14893 



***

### Encode time (seconds since ...)

&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;*NetCDF4 format variable: ``time``*

&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;*Open Refine format variables: `Sampling start date` and `Sampling start time`*

`EncodeTimeCB` converts the HELCOM `time` format to the MARIS `time` format.

In [561]:
#|eval: false
dfs = load_data(fname_in)
tfm = Transformer(dfs, cbs=[LowerStripRdnNameCB(),
                            RemapRdnNameCB(),
                            ParseTimeCB(),
                            EncodeTimeCB(cfg(), verbose = True),
                            CompareDfsAndTfm(dfs)
                            ])
tfm()
print(pd.DataFrame.from_dict(tfm.compare_stats) , '\n')
                            

1 of 37347 entries for `time` are invalid for sediment.
                                                    seawater  sediment  biota
Number of rows in dfs                                  20318     37347  14893
Number of rows in tfm.dfs                              20318     37346  14893
Number of dropped rows                                     0         1      0
Number of rows in tfm.dfs + Number of dropped rows     20318     37347  14893 



In [562]:
tfm.dfs_dropped['sediment'][['YEAR', 'MONTH', 'DAY', 'DATE']]

,YEAR,MONTH,DAY,DATE
35783,NaN,NaN,NaN,NaN


***

### Sanitize value

&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;*NetCDF4 format variable: ``value``.*

&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;*Open Refine format variables: ``Activity or MDA``.*

In [563]:
#| export
# Columns of interest
coi_val = {'seawater' : { 'val' : 'VALUE_Bq/m³'},
                 'biota':  {'val' : 'VALUE_Bq/kg'},
                 'sediment': { 'val' : 'VALUE_Bq/kg'}}

In [564]:
# | export
class SanitizeValue(Callback):
    "Sanitize value by removing blank entries and ensuring the 'value' column is retained."

    def __init__(self, coi: dict):
        """
        Initialize the SanitizeValue callback.

        Args:
            coi (dict): Dictionary containing column names for values based on group.
        """
        fc.store_attr()

    def __call__(self, tfm):
        """
        Sanitize the DataFrames in the transformer by removing rows with blank values in specified columns.

        Args:
            tfm (Transformer): The transformer object containing DataFrames.
        """
        for grp in tfm.dfs.keys():
            self._sanitize_dataframe(tfm.dfs[grp], grp)

    def _sanitize_dataframe(self, df: pd.DataFrame, grp: str):
        """
        Remove rows where specified value columns are blank and ensure the 'value' column is included.

        Args:
            df (pd.DataFrame): DataFrame to sanitize.
            grp (str): Group name to determine column names.
        """
        value_col = self.coi.get(grp, {}).get('val')
        if value_col and value_col in df.columns:
            df.dropna(subset=[value_col], inplace=True)
            # Ensure 'value' column is retained
            if 'value' not in df.columns:
                df['value'] = df[value_col]

In [565]:
#|eval: false
dfs = load_data(fname_in)
tfm = Transformer(dfs, cbs=[LowerStripRdnNameCB(),
                            RemapRdnNameCB(),
                            ParseTimeCB(),
                            EncodeTimeCB(cfg()),                             
                            NormalizeUncUnitCB(),
                            SanitizeValue(coi_val),
                            CompareDfsAndTfm(dfs)
                            ])

tfm()
print(pd.DataFrame.from_dict(tfm.compare_stats) , '\n')

                                                    seawater  sediment  biota
Number of rows in dfs                                  20318     37347  14893
Number of rows in tfm.dfs                              20242     37089  14873
Number of dropped rows                                    76       258     20
Number of rows in tfm.dfs + Number of dropped rows     20318     37347  14893 



***

### Normalize uncertainty

&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;*NetCDF4 format variable: ``uncertainty``.*

&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;*Open Refine format variable: `Uncertainty`.*

Function `unc_rel2stan` coverts uncertainty from relative uncertainty to standard uncertainty.

In [566]:
#| export
# Make measurement and uncertainty units consistent
def unc_rel2stan(df: pd.DataFrame, meas_col: str, unc_col: str) -> pd.Series:
    """
    Convert relative uncertainty to absolute uncertainty.

    Args:
        df (pd.DataFrame): DataFrame containing measurement and uncertainty columns.
        meas_col (str): Name of the column with measurement values.
        unc_col (str): Name of the column with relative uncertainty values (percentages).

    Returns:
        pd.Series: Series with calculated absolute uncertainties.
    """
    return df.apply(lambda row: row[unc_col] * row[meas_col] / 100, axis=1)


For each sample type in the Helcom dataset, the uncertainty is given as a relative uncertainty to the value (i.e., activity). The column names for both the value and the uncertainty vary by sample type. The coi_units_unc dictionary defines the column names for the Value and Uncertainty for each sample type.

In [567]:
#| export
# Columns of interest
coi_units_unc = [('seawater', 'VALUE_Bq/m³', 'ERROR%_m³'),
                 ('biota', 'VALUE_Bq/kg', 'ERROR%'),
                 ('sediment', 'VALUE_Bq/kg', 'ERROR%_kg')]

NormalizeUncUnitCB callback normalizes the uncertainty by converting from relative uncertainty to standard uncertainty. 

In [568]:
class NormalizeUncUnitCB(Callback):
    """Convert from relative error % to uncertainty of activity unit."""
    
    def __init__(self, 
                 fn_convert_unc: Callable[[pd.DataFrame, str, str], pd.Series] = unc_rel2stan,
                 coi: List[Tuple[str, str, str]] = coi_units_unc):
        """
        Initialize the NormalizeUncUnitCB with a function to convert uncertainties and units.

        Args:
            fn_convert_unc (Callable[[pd.DataFrame, str, str], pd.Series], optional): 
                Function to convert relative uncertainty to absolute uncertainty. 
                Defaults to `unc_rel2stan`.
            coi (List[Tuple[str, str, str]], optional): List of tuples with group name, 
                measurement column, and uncertainty column. Defaults to `coi_units_unc`.
        """
        # Automatically initialize attributes using fc.store_attr()
        fc.store_attr()
    
    def __call__(self, tfm: 'Transformer'):
        """
        Apply the conversion function to each DataFrame in the transformer.

        Args:
            tfm (Transformer): The transformer object containing DataFrames.
        """
        for grp, val, unc in self.coi:
            if grp in tfm.dfs:
                df = tfm.dfs[grp]
                df['uncertainty'] = self.fn_convert_unc(df, val, unc)


Apply the transformer for callbacks `LowerStripRdnNameCB`, `RemapRdnNameCB`, `ParseTimeCB` and `NormalizeUncUnitCB()`. Then, print the value (i.e. activity per unit ) and standard uncertainty for each sample type.

In [569]:
#|eval: false
dfs = load_data(fname_in)
tfm = Transformer(dfs, cbs=[LowerStripRdnNameCB(),
                            RemapRdnNameCB(),
                            ParseTimeCB(),
                            EncodeTimeCB(cfg()),                             
                            NormalizeUncUnitCB(),
                            SanitizeValue(coi_val)])

print(tfm()['seawater'][['value', 'uncertainty']][:5])
print(tfm()['biota'][['value', 'uncertainty']][:5])
print(tfm()['sediment'][['value', 'uncertainty']][:5])

   value  uncertainty
0    5.3        1.696
1   19.9        3.980
2   25.5        5.100
3   17.0        4.930
4   22.2        3.996
        value  uncertainty
0    0.010140          NaN
1  135.300000     4.830210
2    0.013980          NaN
3    4.338000     0.150962
4    0.009614          NaN
   value  uncertainty
0   35.0         9.10
1   36.0         7.92
2   38.0         9.12
3   36.0         9.00
4   30.0         6.90


***

### Lookup transformations 

#### Lookup MARIS function 

`get_maris_lut` performs a lookup of data provided in `data_provider_lut` against the MARIS lookup (`maris_lut`) using a fuzzy matching algorithm based on Levenshtein distance. The `get_maris_lut` is used to correct the HELCOM data to a standard format for MARIS. 

In [570]:
def get_maris_lut(fname_in, 
                  fname_cache, # For instance 'species_helcom.pkl'
                  data_provider_lut: str, # Data provider lookup table name
                  data_provider_id_col: str, # Data provider lookup column id of interest
                  data_provider_name_col: str, # Data provider lookup column name of interest
                  maris_lut: Callable, # Function retrieving MARIS source lookup table
                  maris_id: str, # Id of MARIS lookup table nomenclature item to match
                  maris_name: str, # Name of MARIS lookup table nomenclature item to match
                  unmatched_fixes: dict = {},
                  as_dataframe: bool = False,
                  overwrite: bool = False
                 ):
    # Use fname_cache directly for the file path
    cache_file = cache_path() / fname_cache
    lut = {}
    maris_lut = maris_lut()
    df = pd.read_csv(Path(fname_in) / data_provider_lut)
    
    if overwrite or (not cache_file.exists()):
        for _, row in tqdm(df.iterrows(), total=len(df), desc="Processing"):
            # Fix if unmatched
            has_to_be_fixed = row[data_provider_id_col] in unmatched_fixes            
            name_to_match = unmatched_fixes[row[data_provider_id_col]] if has_to_be_fixed else row[data_provider_name_col]

            # Match
            result = match_maris_lut(maris_lut, name_to_match, maris_id, maris_name)
            match = Match(result.iloc[0][maris_id], result.iloc[0][maris_name], 
                          row[data_provider_name_col], result.iloc[0]['score'])
            
            lut[row[data_provider_id_col]] = match
        
        fc.save_pickle(cache_file, lut)
    else:
        lut = fc.load_pickle(cache_file)

    if as_dataframe:
        df_lut = pd.DataFrame({k: asdict(v) for k, v in lut.items()}).transpose()
        df_lut.index.name = 'source_id'
        return df_lut.sort_values(by='match_score', ascending=False)
    else:
        return lut


#### Lookup : Biota species

&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;*NetCDF4 format variable: ``species``.*

&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;*Open Refine format variable: `Species`.*

The HELCOM dataset includes look-up in the `RUBIN_NAME.csv` file for biota species. 

In [571]:
#|eval: false
df_rubin = pd.read_csv(Path(fname_in) / 'RUBIN_NAME.csv')
df_rubin.head(5)

,RUBIN_ID,RUBIN,SCIENTIFIC NAME,ENGLISH NAME
0,11,ABRA BRA,ABRAMIS BRAMA,BREAM
1,12,ANGU ANG,ANGUILLA ANGUILLA,EEL
2,13,ARCT ISL,ARCTICA ISLANDICA,ISLAND CYPRINE
3,14,ASTE RUB,ASTERIAS RUBENS,COMMON STARFISH
4,15,CARD EDU,CARDIUM EDULE,COCKLE


Create `unmatched_fixes_biota_species` to correct the spelling of names that are unmatched in the HELCOM dataset. 

In [572]:
#|export
unmatched_fixes_biota_species = {
    'CARD EDU': 'Cerastoderma edule',
    'LAMI SAC': 'Saccharina latissima',
    'PSET MAX': 'Scophthalmus maximus',
    'STIZ LUC': 'Sander luciopercas'}

In [573]:
#|eval: false
species_lut_df = get_maris_lut(fname_in, 
                               fname_cache='species_helcom.pkl', 
                               data_provider_lut='RUBIN_NAME.csv',
                               data_provider_id_col='RUBIN',
                               data_provider_name_col='SCIENTIFIC NAME',
                               maris_lut=species_lut_path,
                               maris_id='species_id',
                               maris_name='species',
                               unmatched_fixes=unmatched_fixes_biota_species,
                               as_dataframe=True,
                               overwrite=True)

Processing:   0%|          | 0/43 [00:00<?, ?it/s]

Processing: 100%|██████████| 43/43 [00:07<00:00,  5.53it/s]


Display `species_lut_df`. The `match_score` represents the number insertions, deletions, or substitutions needed to transform from the HECOM source name (`source_name`) to the maris name, (`matched_maris_name`). 

In [574]:
#|eval: false
species_lut_df.head()

,matched_id,matched_maris_name,source_name,match_score
source_id,,,,
ENCH CIM,276,Echinodermata,ENCHINODERMATA CIM,5
MACO BAL,122,Macoma balthica,MACOMA BALTICA,1
STUC PEC,704,Stuckenia pectinata,STUCKENIA PECTINATE,1
STIZ LUC,285,Sander lucioperca,STIZOSTEDION LUCIOPERCA,1
POLY FUC,245,Polysiphonia fucoides,POLYSIPHONIA FUCOIDES,0


Show `species_lut_df` where `match_type` is not a perfect match ( i.e. not equal 0).

In [575]:
species_lut_df[species_lut_df['match_score'] >= 1]

,matched_id,matched_maris_name,source_name,match_score
source_id,,,,
ENCH CIM,276,Echinodermata,ENCHINODERMATA CIM,5
MACO BAL,122,Macoma balthica,MACOMA BALTICA,1
STUC PEC,704,Stuckenia pectinata,STUCKENIA PECTINATE,1
STIZ LUC,285,Sander lucioperca,STIZOSTEDION LUCIOPERCA,1


`LookupBiotaSpeciesCB` applies the corrected `biota` `species` data obtained from the `get_maris_lut` function to the `biota` dataframe in the dictionary of dataframes, `dfs`.

In [576]:
class LookupBiotaSpeciesCB(Callback):
    """
    Biota species remapped to MARIS db:
        CARD EDU: Cerastoderma edule
        LAMI SAC: Saccharina latissima
        PSET MAX: Scophthalmus maximus
        STIZ LUC: Sander luciopercas
    """
    def __init__(self, fn_lut: Callable[[], dict]):
        """
        Initialize the LookupBiotaSpeciesCB with a function to generate the lookup table.

        Args:
            fn_lut (Callable[[], dict]): Function that returns the lookup table dictionary.
        """
        fc.store_attr()

    def __call__(self, tfm: 'Transformer'):
        """
        Remap biota species names in the DataFrame using the lookup table and print unmatched RUBIN values.

        Args:
            tfm (Transformer): The transformer object containing DataFrames.
        """
        lut = self.fn_lut()
        
        # Apply the function to the 'RUBIN' column
        tfm.dfs['biota']['species'] = tfm.dfs['biota']['RUBIN'].apply(lambda x: self._get_species(x, lut))

    def _get_species(self, rubin_value: str, lut: dict):
        """
        Get the matched_id from the lookup table and print RUBIN if the matched_id is -1.

        Args:
            rubin_value (str): The RUBIN value from the DataFrame.
            lut (dict): The lookup table dictionary.

        Returns:
            The matched_id from the lookup table.
        """
        match = lut.get(rubin_value.strip(), Match(-1, None, None, None))
        if match.matched_id == -1:
            self.print_unmatched_rubin(rubin_value)
        return match.matched_id

    def print_unmatched_rubin(self, rubin_value: str):
        """
        Print the RUBIN value if the matched_id is -1.

        Args:
            rubin_value (str): The RUBIN value from the DataFrame.
        """
        print(f"Unmatched RUBIN: {rubin_value}")


`get_maris_species` defines a partial function of `get_maris_lut`, with predefined arguments  for species lookup.

In [577]:
#| export
get_maris_species = partial(get_maris_lut,
                            fname_in, fname_cache='species_helcom.pkl', 
                            data_provider_lut='RUBIN_NAME.csv',
                            data_provider_id_col='RUBIN',
                            data_provider_name_col='SCIENTIFIC NAME',
                            maris_lut=species_lut_path,
                            maris_id='species_id',
                            maris_name='species',
                            unmatched_fixes=unmatched_fixes_biota_species,
                            as_dataframe=False,
                            overwrite=False)

Apply the transformer for callbacks `LowerStripRdnNameCB`, `RemapRdnNameCB`, `ParseTimeCB`,  `NormalizeUncUnitCB()` and `LookupBiotaSpeciesCB(get_maris_species)`. Then, print the unique `species` for the `biota` dataframe.

In [578]:
#|eval: false
dfs = load_data(fname_in)
tfm = Transformer(dfs, cbs=[LowerStripRdnNameCB(),
                            RemapRdnNameCB(),
                            ParseTimeCB(),
                            EncodeTimeCB(cfg()),                             
                            NormalizeUncUnitCB(),
                            SanitizeValue(coi_val),                       
                            LookupBiotaSpeciesCB(get_maris_species)
                            ])

#print(tfm()['biota'][['RUBIN', 'species']][:10])
print(tfm()['biota']['species'].unique())

[  99  243   50  139  270  192  191  284   84  269  122   96  287  279
  278  288  286  244  129  275  271  285  283  247  120   59  280  274
  273  290  289  272  277  276   21  282  110  281  245  704 1524]


***

#### Lookup : Biota tissues

&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;*NetCDF4 format variable: ``body_part``.*

&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;*Open Refine format variable: `Body part`.*

The HELCOM dataset includes look-up in the `TISSUE.csv` file for biota tissues. Biota tissue is known as `body part` in the maris data set.    

In [579]:
#|eval: false
pd.read_csv('../../_data/accdb/mors/csv/TISSUE.csv').head()

,TISSUE,TISSUE_DESCRIPTION
0,1,WHOLE FISH
1,2,WHOLE FISH WITHOUT ENTRAILS
2,3,WHOLE FISH WITHOUT HEAD AND ENTRAILS
3,4,FLESH WITH BONES
4,5,FLESH WITHOUT BONES (FILETS)


Create `unmatched_fixes_biota_tissues` to correct entries in the HELCOM dataset. 

In [580]:
#|export
unmatched_fixes_biota_tissues = {
    3: 'Whole animal eviscerated without head',
    12: 'Viscera',
    8: 'Skin'}

In [581]:
#|eval: false
tissues_lut_df = get_maris_lut(fname_in, 
                               fname_cache='tissues_helcom.pkl', 
                               data_provider_lut='TISSUE.csv',
                               data_provider_id_col='TISSUE',
                               data_provider_name_col='TISSUE_DESCRIPTION',
                               maris_lut=bodyparts_lut_path,
                               maris_id='bodypar_id',
                               maris_name='bodypar',
                               unmatched_fixes=unmatched_fixes_biota_tissues,
                               as_dataframe=True,
                               overwrite=True)

Processing:   0%|          | 0/29 [00:00<?, ?it/s]

Processing: 100%|██████████| 29/29 [00:00<00:00, 93.70it/s]


In [582]:
tissues_lut_df.head()

,matched_id,matched_maris_name,source_name,match_score
source_id,,,,
2,52,Flesh without bones,WHOLE FISH WITHOUT ENTRAILS,13
5,52,Flesh without bones,FLESH WITHOUT BONES (FILETS),9
1,1,Whole animal,WHOLE FISH,5
15,53,Stomach and intestine,STOMACH + INTESTINE,3
41,1,Whole animal,WHOLE ANIMALS,1


`LookupBiotaBodyPartCB` applies the corrected `biota` `TISSUE` data obtained from the `get_maris_lut` function to the `biota` dataframe in the dictionary of dataframes, `dfs`.

In [583]:
#| export
class LookupBiotaBodyPartCB(Callback):
    """
    Update bodypart id based on MARIS dbo_bodypar.xlsx:
        - 3: 'Whole animal eviscerated without head',
        - 12: 'Viscera',
        - 8: 'Skin'
    """
    def __init__(self, fn_lut: Callable[[], dict]):
        """
        Initialize the LookupBiotaBodyPartCB with a function to generate the lookup table.

        Args:
            fn_lut (Callable[[], dict]): Function that returns the lookup table dictionary.
        """
        fc.store_attr()

    def __call__(self, tfm: 'Transformer'):
        """
        Remap biota body parts in the DataFrame using the lookup table and print unmatched TISSUE values.

        Args:
            tfm (Transformer): The transformer object containing DataFrames.
        """
        lut = self.fn_lut()
        tfm.dfs['biota']['body_part'] = tfm.dfs['biota']['TISSUE'].apply(lambda x: self._get_body_part(x, lut))

    def _get_body_part(self, tissue_value: str, lut: dict):
        """
        Get the matched_id from the lookup table and print TISSUE if the matched_id is -1.

        Args:
            tissue_value (str): The TISSUE value from the DataFrame.
            lut (dict): The lookup table dictionary.

        Returns:
            The matched_id from the lookup table.
        """
        match = lut.get(tissue_value, Match(-1, None, None, None))
        if match.matched_id == -1:
            self.print_unmatched_tissue(tissue_value)
        return match.matched_id

    def print_unmatched_tissue(self, tissue_value: str):
        """
        Print the TISSUE value if the matched_id is -1.

        Args:
            tissue_value (str): The TISSUE value from the DataFrame.
        """
        print(f"Unmatched TISSUE: {tissue_value}")


`get_maris_bodypart` defines a partial function of `get_maris_lut`, with predefined arguments  for  `TISSUE` (or `bodypar`) lookup.

In [584]:
#| export
get_maris_bodypart = partial(get_maris_lut,
                             fname_in,
                             fname_cache='tissues_helcom.pkl', 
                             data_provider_lut='TISSUE.csv',
                             data_provider_id_col='TISSUE',
                             data_provider_name_col='TISSUE_DESCRIPTION',
                             maris_lut=bodyparts_lut_path,
                             maris_id='bodypar_id',
                             maris_name='bodypar',
                             unmatched_fixes=unmatched_fixes_biota_tissues)

Apply the transformer for callbacks `LowerStripRdnNameCB`, `RemapRdnNameCB`, `ParseTimeCB`,  `NormalizeUncUnitCB()`, `LookupBiotaSpeciesCB(get_maris_species)` and `LookupbioooooooootaBodyPartCB(get_maris_bodypart)`. Then, print the `TISSUE` and `body_part` for the `biota` dataframe.

In [585]:
#|eval: false
dfs = load_data(fname_in)
tfm = Transformer(dfs, cbs=[LowerStripRdnNameCB(),
                            RemapRdnNameCB(),
                            ParseTimeCB(),
                            EncodeTimeCB(cfg()),                             
                            NormalizeUncUnitCB(),
                            SanitizeValue(coi_val),                       
                            LookupBiotaSpeciesCB(get_maris_species),
                            LookupBiotaBodyPartCB(get_maris_bodypart)
                            ])

print(tfm()['biota'][['TISSUE', 'body_part']][:5])

   TISSUE  body_part
0       5         52
1       5         52
2       5         52
3       5         52
4       5         52


***

#### Lookup : Biogroup

&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;*NetCDF4 format variable: ``bio_group``.*

&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;*Open Refine format variable: Biogroup is not included.*

`get_biogroup_lut` reads the file at `species_lut_path()` and from the contents of this file creates a dictionary linking `species_id` to `biogroup_id`.

In [586]:
#| export
def get_biogroup_lut(maris_lut: str) -> dict:
    """
    Retrieve a lookup table for biogroup ids from a MARIS lookup table.

    Args:
        maris_lut (str): Path to the MARIS lookup table (Excel file).

    Returns:
        dict: A dictionary mapping species_id to biogroup_id.
    """
    species = pd.read_excel(maris_lut)
    return species[['species_id', 'biogroup_id']].set_index('species_id').to_dict()['biogroup_id']


`LookupBiogroupCB` applies the corrected `biota` `bio group` data obtained from the `get_maris_lut` function to the `biota` dataframe in the dictionary of dataframes, `dfs`.

In [587]:
#| export
class LookupBiogroupCB(Callback):
    """
    Update biogroup id based on MARIS dbo_species.xlsx
    """
    def __init__(self, fn_lut: Callable[[], dict]):
        """
        Initialize the LookupBiogroupCB with a function to generate the lookup table.

        Args:
            fn_lut (Callable[[], dict]): Function that returns the lookup table dictionary.
        """
        fc.store_attr()

    def __call__(self, tfm: 'Transformer'):
        """
        Update the 'bio_group' column in the DataFrame using the lookup table and print unmatched species values.

        Args:
            tfm (Transformer): The transformer object containing DataFrames.
        """
        lut = self.fn_lut()
        tfm.dfs['biota']['bio_group'] = tfm.dfs['biota']['species'].apply(lambda x: self._get_biogroup(x, lut))

    def _get_biogroup(self, species_value: str, lut: dict) -> int:
        """
        Get the biogroup id from the lookup table and print species if the biogroup id is not found.

        Args:
            species_value (str): The species value from the DataFrame.
            lut (dict): The lookup table dictionary.

        Returns:
            int: The biogroup id from the lookup table.
        """
        biogroup_id = lut.get(species_value, -1)
        if biogroup_id == -1:
            self.print_unmatched_species(species_value)
        return biogroup_id

    def print_unmatched_species(self, species_value: str):
        """
        Print the species value if the biogroup id is not found.

        Args:
            species_value (str): The species value from the DataFrame.
        """
        print(f"Unmatched species: {species_value}")


Apply the transformer for callbacks `LowerStripRdnNameCB`, `RemapRdnNameCB`, `ParseTimeCB`,  `NormalizeUncUnitCB()`, `LookupBiotaSpeciesCB(get_maris_species)`, `LookupBiotaBodyPartCB(get_maris_bodypart)`, `LookupSedimentCB(get_maris_sediments)` and `LookupBiogroupCB(partial(get_biogroup_lut, species_lut_path())` . Then, print the `bio_group` for the `biota` dataframe.

In [588]:

#|eval: false
dfs = load_data(fname_in)
tfm = Transformer(dfs, cbs=[LowerStripRdnNameCB(),
                            RemapRdnNameCB(),
                            ParseTimeCB(),
                            EncodeTimeCB(cfg()),                             
                            NormalizeUncUnitCB(),
                            SanitizeValue(coi_val),                       
                            LookupBiotaSpeciesCB(get_maris_species),
                            LookupBiotaBodyPartCB(get_maris_bodypart),                          
                            LookupBiogroupCB(partial(get_biogroup_lut, species_lut_path()))
                            ])

print(tfm()['biota']['bio_group'].unique())

[ 4  2 14 11  8  3]


***

#### Lookup : Sediment types

The HELCOM dataset includes look-up in the `SEDIMENT_TYPE.csv` file for Sediment types. 

&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;*NetCDF4 format variable: ``sed_type``.*

&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;*Open Refine format variable: `Sediment type`.*

In [589]:
#|eval: false
df_sediment = pd.read_csv(Path(fname_in) / 'SEDIMENT_TYPE.csv')
df_sediment.head()

,SEDI,SEDIMENT TYPE,RECOMMENDED TO BE USED
0,-99,NO DATA,NaN
1,30,SILT AND GRAVEL,YES
2,0,GRAVEL,YES
3,1,SAND,YES
4,2,FINE SAND,NO


Create `unmatched_fixes_sediments` to correct entries in the HELCOM dataset. 

In [590]:
#|export
unmatched_fixes_sediments = {
    #np.nan: 'Not applicable',
    -99: '(Not available)'
}

In [591]:
#|eval: false
sediments_lut_df = get_maris_lut(
    fname_in, 
    fname_cache='sediments_helcom.pkl', 
    data_provider_lut='SEDIMENT_TYPE.csv',
    data_provider_id_col='SEDI',
    data_provider_name_col='SEDIMENT TYPE',
    maris_lut=sediments_lut_path,
    maris_id='sedtype_id',
    maris_name='sedtype',
    unmatched_fixes=unmatched_fixes_sediments,
    as_dataframe=True,
    overwrite=True)

Processing: 100%|██████████| 47/47 [00:00<00:00, 100.91it/s]


`get_maris_sediments` defines a partial function of `get_maris_lut`, with predefined arguments  for  `SEDI` (or `sedtype`) lookup.

In [592]:
#| export
get_maris_sediments = partial(
    get_maris_lut,
    fname_in, 
    fname_cache='sediments_helcom.pkl', 
    data_provider_lut='SEDIMENT_TYPE.csv',
    data_provider_id_col='SEDI',
    data_provider_name_col='SEDIMENT TYPE',
    maris_lut=sediments_lut_path,
    maris_id='sedtype_id',
    maris_name='sedtype',
    unmatched_fixes=unmatched_fixes_sediments)

`LookupSedimentCB` applies the corrected `sediment` `SEDI` data obtained from the `get_maris_lut` function to the `sediment` dataframe in the dictionary of dataframes, `dfs`.

In [593]:
#| export
def preprocess_sedi(df, column_name='SEDI'):
    """
    Preprocess the 'SEDI' column in the DataFrame by handling missing values and specific replacements.

    Args:
        df (pd.DataFrame): The DataFrame containing the 'SEDI' column.
        column_name (str): The name of the column to preprocess. Default is 'SEDI'.
    
    Returns:
        pd.DataFrame: The DataFrame with preprocessed 'SEDI' column.
    """
    if column_name in df.columns:
        df[column_name] = df[column_name].fillna(-99).astype('int')
        df[column_name].replace([56, 73], -99, inplace=True)
    return df


In [594]:
#| export
class LookupSedimentCB(Callback):
    """
    Update sediment id based on MARIS dbo_sedtype.xlsx.
    """
    def __init__(self, fn_lut: Callable[[], dict], preprocess_fn: Callable[[pd.DataFrame, str], pd.DataFrame] = preprocess_sedi):
        """
        Initialize the LookupSedimentCB with a function to generate the lookup table and a preprocessing function.

        Args:
            fn_lut (Callable[[], dict]): Function that returns the lookup table dictionary.
            preprocess_fn (Callable[[pd.DataFrame, str], pd.DataFrame]): Function to preprocess the sediment DataFrame. Default is preprocess_sedi.
        """
        fc.store_attr()
        self.preprocess_fn = preprocess_fn

    def __call__(self, tfm: 'Transformer'):
        """
        Remap sediment types in the DataFrame using the lookup table and handle specific replacements.

        Args:
            tfm (Transformer): The transformer object containing DataFrames.
        """
        lut = self.fn_lut()

        # Apply preprocessing to the 'SEDI' column
        tfm.dfs['sediment'] = self.preprocess_fn(tfm.dfs['sediment'])
        
        # Apply the lookup function
        tfm.dfs['sediment']['sed_type'] = tfm.dfs['sediment']['SEDI'].apply(lambda x: self._get_sediment_type(x, lut))

    def _get_sediment_type(self, sedi_value: int, lut: dict):
        """
        Get the matched_id from the lookup table and print SEDI if the matched_id is -1.

        Args:
            sedi_value (int): The SEDI value from the DataFrame.
            lut (dict): The lookup table dictionary.

        Returns:
            The matched_id from the lookup table.
        """
        match = lut.get(sedi_value, Match(-1, None, None, None))
        if match.matched_id == -1:
            self._print_unmatched_sedi(sedi_value)
        return match.matched_id

    def _print_unmatched_sedi(self, sedi_value: int):
        """
        Print the SEDI value if the matched_id is -1.

        Args:
            sedi_value (int): The SEDI value from the DataFrame.
        """
        print(f"Unmatched SEDI: {sedi_value}")


Apply the transformer for callbacks `LowerStripRdnNameCB`, `RemapRdnNameCB`, `ParseTimeCB`,  `NormalizeUncUnitCB()`, `LookupBiotaSpeciesCB(get_maris_species)`, `LookupBiotaBodyPartCB(get_maris_bodypart)` and `LookupSedimentCB(get_maris_sediments)`. Then, print the `SEDI` and `sed_type` for the `biota` dataframe.

In [595]:

#|eval: false
dfs = load_data(fname_in)
tfm = Transformer(dfs, cbs=[LowerStripRdnNameCB(),
                            RemapRdnNameCB(),
                            ParseTimeCB(),
                            EncodeTimeCB(cfg()),                             
                            NormalizeUncUnitCB(),
                            SanitizeValue(coi_val),                       
                            LookupBiotaSpeciesCB(get_maris_species),
                            LookupBiotaBodyPartCB(get_maris_bodypart),                          
                            LookupBiogroupCB(partial(get_biogroup_lut, species_lut_path())),
                            LookupSedimentCB(get_maris_sediments)
                            ])

tfm()
print(tfm.dfs['sediment'][['SEDI', 'sed_type']][:5])

   SEDI  sed_type
0   -99         0
1   -99         0
2   -99         0
3   -99         0
4   -99         0


***

#### Lookup : Units

&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;*NetCDF4 format variable: ``unit``.*

&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;*Open Refine format variable: ``Unit``.*

Create `renaming_unit_rules` to rename the units. 

In [596]:
#| export
# Define unit names renaming rules
renaming_unit_rules = {
    'seawater': 1,  # 'Bq/m3'
    'sediment': 4,  # 'Bq/kgd' for sediment
    'biota': {
        'D': 4,  # 'Bq/kgd'
        'W': 5,  # 'Bq/kgw'
        'F': 5   # 'Bq/kgw' (assumed to be 'Fresh', so set to wet)
    }
}


`LookupUnitCB` defines a `unit` column each dataframe based on the units provided in the value (`VALUE_Bq/m³` or `VALUE_Bq/kg`) column of the HELCOM dataset. 

In [597]:
#| export
class LookupUnitCB(Callback):
    def __init__(self, renaming_unit_rules=renaming_unit_rules):
        """
        Initialize the LookupUnitCB with unit renaming rules.

        Args:
            renaming_unit_rules (dict): Dictionary containing renaming rules for different unit categories.
        """
        fc.store_attr()

    def __call__(self, tfm: 'Transformer'):
        """
        Apply unit renaming rules to DataFrames within the transformer.

        Args:
            tfm (Transformer): The transformer object containing DataFrames.
        """
        for grp in tfm.dfs:
            rules = renaming_unit_rules.get(grp)
            if rules is not None:
                # if group tules include a dictionary, apply the dictionay. 
                if isinstance(rules, dict):
                    # Apply rules based on the 'BASIS' column
                    tfm.dfs[grp]['unit'] = tfm.dfs[grp]['BASIS'].apply(lambda x: rules.get(x, 0))
                else:
                    # Apply a single rule to the entire DataFrame
                    tfm.dfs[grp]['unit'] = rules


Apply the transformer for callbacks `LowerStripRdnNameCB`, `RemapRdnNameCB`, `ParseTimeCB`,  `NormalizeUncUnitCB()`, `LookupBiotaSpeciesCB(get_maris_species)`, `LookupBiotaBodyPartCB(get_maris_bodypart)`, `LookupSedimentCB(get_maris_sediments)`, `LookupBiogroupCB(partial(get_biogroup_lut, species_lut_path())` and `LookupUnitCB()`. Then, print the unique `unit` for the `seawater` dataframe.

In [598]:
#|eval: false
dfs = load_data(fname_in)
tfm = Transformer(dfs, cbs=[LowerStripRdnNameCB(),
                            RemapRdnNameCB(),
                            ParseTimeCB(),
                            EncodeTimeCB(cfg()),                             
                            NormalizeUncUnitCB(),
                            SanitizeValue(coi_val),                       
                            LookupBiotaSpeciesCB(get_maris_species),
                            LookupBiotaBodyPartCB(get_maris_bodypart),                          
                            LookupBiogroupCB(partial(get_biogroup_lut, species_lut_path())),
                            LookupSedimentCB(get_maris_sediments),
                            LookupUnitCB()])

print(tfm()['biota']['unit'].unique())

[5 0 4]


***

#### Lookup : Detection limit or Value type

&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;*NetCDF4 format variable: ``detection_limit``.*

&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;*Open Refine foramt variable: ``Value type``.*

Create `coi_dl` to define the column names related to Value type for each dataset. 

In [599]:
#| export
# Columns of interest
coi_dl = {'seawater' : { 'val' : 'VALUE_Bq/m³',
                        'unc' : 'ERROR%_m³',
                        'dl' : '< VALUE_Bq/m³'},
                 'biota':  {'val' : 'VALUE_Bq/kg',
                            'unc' : 'ERROR%',
                            'dl' : '< VALUE_Bq/kg'},
                 'sediment': { 'val' : 'VALUE_Bq/kg',
                              'unc' : 'ERROR%_kg',
                              'dl' : '< VALUE_Bq/kg'}}

`get_detectionlimit_lut` reads the file at `detection_limit_lut_path()` and from the contents of this file creates a dictionary linking `name` to `id`.
| id | name | name_sanitized |
| :-: | :-: | :-: |
|-1|Not applicable|Not applicable|
|0|Not Available|Not available|
|1|=|Detected value|
|2|<|Detection limit|
|3|ND|Not detected|
|4|DE|Derived|

In [600]:
#| export 
def get_detectionlimit_lut():
    df = pd.read_excel(detection_limit_lut_path(), usecols=['name','id'])
    return df.set_index('name').to_dict()['id']

`LookupDetectionLimitCB` creates a `detection_limit` column with values determined as follows:
1. Perform a lookup with the appropriate columns value type (or detection limit) columns (`< VALUE_Bq/m³` or `< VALUE_Bq/kg`) against the table returned from the function `get_detectionlimit_lut`.
2. If `< VALUE_Bq/m³` or `< VALUE_Bq/kg>` is NaN but both activity values (`VALUE_Bq/m³` or `VALUE_Bq/kg`) and standard uncertainty (`ERROR%_m³`, `ERROR%`, or `ERROR%_kg`) are provided, then assign the ID of `1` (i.e. "Detected value").
3. For other NaN values in the `detection_limit` column, set them to `0` (i.e. `Not Available`).

In [601]:
# | export
class LookupDetectionLimitCB(Callback):
    "Remap value type to MARIS format."

    def __init__(self, 
                 coi=coi_dl,
                 fn_lut=get_detectionlimit_lut):
        """
        Initialize the LookupDetectionLimitCB with configuration options and lookup function.

        Args:
            coi (dict): Configuration options for column names.
            fn_lut (Callable): Function that returns a lookup table.
        """
        fc.store_attr()

    def __call__(self, tfm: 'Transformer'):
        """
        Remap detection limits in the DataFrames using the lookup table.

        Args:
            tfm (Transformer): The transformer object containing DataFrames.
        """
        lut = self.fn_lut()
        
        for grp in tfm.dfs:
            df = tfm.dfs[grp]
            self._update_detection_limit(df, grp, lut)
    
    def _update_detection_limit(self, df: pd.DataFrame, grp: str, lut: dict):
        """
        Update detection limit column in the DataFrame based on lookup table and rules.

        Args:
            df (pd.DataFrame): The DataFrame to modify.
            grp (str): The group name to get the column configuration.
            lut (dict): The lookup table dictionary.
        """
        detection_col = self.coi[grp]['dl']
        value_col = self.coi[grp]['val']
        uncertainty_col = self.coi[grp]['unc']
        
        # Copy detection limit column
        df['detection_limit'] = df[detection_col]
        
        # Fill values with '=' or 'Not Available'
        condition = ((df[value_col].notna()) & (df[uncertainty_col].notna()) &
                     (~df['detection_limit'].isin(lut.keys())))
        df.loc[condition, 'detection_limit'] = '='
        df.loc[~df['detection_limit'].isin(lut.keys()), 'detection_limit'] = 'Not Available'
        
        # Perform lookup
        df['detection_limit'] = df['detection_limit'].map(lut)


Apply the transformer for callbacks `LowerStripRdnNameCB`, `RemapRdnNameCB`, `ParseTimeCB`,  `NormalizeUncUnitCB()`, `LookupBiotaSpeciesCB(get_maris_species)`, `LookupBiotaBodyPartCB(get_maris_bodypart)`, `LookupSedimentCB(get_maris_sediments)`, `LookupBiogroupCB(partial(get_biogroup_lut, species_lut_path())`, `LookupUnitCB()` and `LookupDetectionLimitCB`. Then, print the unique `detection_limit` for the `seawater` dataframe.

In [602]:
#|eval: false
dfs = load_data(fname_in)
tfm = Transformer(dfs, cbs=[LowerStripRdnNameCB(),
                            RemapRdnNameCB(),
                            ParseTimeCB(),
                            EncodeTimeCB(cfg()),                             
                            NormalizeUncUnitCB(),
                            SanitizeValue(coi_val),                       
                            LookupBiotaSpeciesCB(get_maris_species),
                            LookupBiotaBodyPartCB(get_maris_bodypart),                          
                            LookupBiogroupCB(partial(get_biogroup_lut, species_lut_path())),
                            LookupSedimentCB(get_maris_sediments),
                            LookupUnitCB(),
                            LookupDetectionLimitCB()])

print(tfm()['seawater']['detection_limit'].unique())

[1 2 0]


***

#### Lookup : Method

&nbsp;&nbsp;&nbsp;&nbsp;&nbsp; *NetCDF4 format variables: ``counting_method``, ``sampling_method`` and ``preparation_method``.*

&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;*Open Refine format variable: ``Sampling method``,	``Preparation method`` and ``Counting method``.*

> 'Method' is provided in the HELCOM data but some work is required to link it to MARIS 'counting_method', 'sampling_method' and 'preparation_method'.

***

### Data provider sample id

&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;*NetCDF4 format variable: ``data_provider_sample_id``*

&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;*Open Refine format variable: ``Sample ID``*

>  MARIS NetCDF4 format for variable type ``data_provider_sample_id`` does not support vlen strings.

In [603]:
# | export
class RemapDataProviderSampleIdCB(Callback):
    """
    Remap 'KEY' column to 'data_provider_sample_id' in each DataFrame.
    """

    def __init__(self):
        """
        Initialize the RemapDataProviderSampleIdCB.
        """
        fc.store_attr()

    def __call__(self, tfm: 'Transformer'):
        """
        Remap 'KEY' column to 'data_provider_sample_id' in the DataFrames.

        Args:
            tfm (Transformer): The transformer object containing DataFrames.
        """
        for grp in tfm.dfs:
            self._remap_sample_id(tfm.dfs[grp])
    
    def _remap_sample_id(self, df: pd.DataFrame):
        """
        Remap the 'KEY' column to 'data_provider_sample_id' in the DataFrame.

        Args:
            df (pd.DataFrame): The DataFrame to modify.
        """
        df['data_provider_sample_id'] = df['KEY']


In [604]:
#|eval: false
dfs = load_data(fname_in)
tfm = Transformer(dfs, cbs=[LowerStripRdnNameCB(),
                            RemapRdnNameCB(),
                            ParseTimeCB(),
                            EncodeTimeCB(cfg()),                             
                            NormalizeUncUnitCB(),
                            SanitizeValue(coi_val),                       
                            LookupBiotaSpeciesCB(get_maris_species),
                            LookupBiotaBodyPartCB(get_maris_bodypart),                          
                            LookupBiogroupCB(partial(get_biogroup_lut, species_lut_path())),
                            LookupSedimentCB(get_maris_sediments),
                            LookupUnitCB(),
                            LookupDetectionLimitCB(),
                            RemapDataProviderSampleIdCB(),
                            CompareDfsAndTfm(dfs)
                            ])

print(tfm()['seawater']['data_provider_sample_id'].unique())
print(pd.DataFrame.from_dict(tfm.compare_stats) , '\n')


['WKRIL2012003' 'WKRIL2012004' 'WKRIL2012005' ... 'WSSSM2018006'
 'WSSSM2018007' 'WSSSM2018008']
                                                    seawater  sediment  biota
Number of rows in dfs                                  20318     37347  14893
Number of rows in tfm.dfs                              20242     37089  14873
Number of dropped rows                                    76       258     20
Number of rows in tfm.dfs + Number of dropped rows     20318     37347  14893 



***

### Filtered

&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;*NetCDF4 format variable: ``filtered``*

&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;*Open Refine format variable: ``Filtered``*

`get_filtered_lut` reads the file at `filtered_lut_path()` and from the contents of this file creates a dictionary linking `name` to `id`.

In [667]:
# | export
def get_filtered_lut() -> dict:
    """
    Retrieve a filtered lookup table from an Excel file.

    Returns:
        dict: A dictionary mapping names to IDs.
    """
    df = pd.read_excel(filtered_lut_path(), usecols=['name', 'id'])
    return df.set_index('name').to_dict()['id']


In [668]:
get_filtered_lut()

{'Not applicable': -1, 'Not available': 0, 'Yes': 1, 'No': 2}

Create  `renaming_rules` to rename the HELCOM data to the MARIS format.

In [669]:
renaming_rules = {'N': 'No',
                  'n': 'No',
                  'F': 'Yes'}

`LookupFiltCB` converts the HELCOM `FILT` format to the MARIS `FILT` format.

In [672]:
# | export
class LookupFiltCB(Callback):
    "Lookup FILT value."
    
    def __init__(self,
                 rules=renaming_rules,
                 fn_lut=get_filtered_lut):
        """
        Initialize the LookupFiltCB with renaming rules and a function for generating the lookup table.

        Args:
            rules (dict): Dictionary mapping FILT codes to their corresponding names.
            fn_lut (Callable[[], dict]): Function that returns the lookup table dictionary.
        """
        fc.store_attr()

    def __call__(self, tfm: 'Transformer'):
        """
        Update the FILT column in the DataFrames using the renaming rules and lookup table.

        Args:
            tfm (Transformer): The transformer object containing DataFrames.
        """
        lut = self.fn_lut()
        rules = self.rules
        
        for grp in tfm.dfs.keys():
            if "FILT" in tfm.dfs[grp].columns:
                self._update_filt_column(tfm.dfs[grp], rules, lut)

    def _update_filt_column(self, df: pd.DataFrame, rules: dict, lut: dict):
        """
        Update the FILT column based on renaming rules and lookup table.

        Args:
            df (pd.DataFrame): The DataFrame to modify.
            rules (dict): Dictionary mapping FILT codes to their corresponding names.
            lut (dict): Dictionary for lookup values.
        """
        # Fill values that are not in the renaming rules with 'Not available'.
        df['FILT'] = df['FILT'].apply(lambda x: rules.get(x, 'Not available'))
        
        # Perform lookup
        df['FILT'] = df['FILT'].map(lambda x: lut.get(x, 0))


Apply the transformer for callbacks `LowerStripRdnNameCB`, `RemapRdnNameCB`, `ParseTimeCB`,  `NormalizeUncUnitCB()`, `LookupBiotaSpeciesCB(get_maris_species)`, `LookupBiotaBodyPartCB(get_maris_bodypart)`, `LookupSedimentCB(get_maris_sediments)`, `LookupBiogroupCB(partial(get_biogroup_lut, species_lut_path())`, `LookupUnitCB()`,  `LookupDetectionLimitCB` and `LookupFiltCB()`. Then, print the unique `FILT` for the `seawater` dataframe.

In [673]:
#|eval: false
dfs = load_data(fname_in)
tfm = Transformer(dfs, cbs=[LowerStripRdnNameCB(),
                            RemapRdnNameCB(),
                            ParseTimeCB(),
                            EncodeTimeCB(cfg()),                             
                            NormalizeUncUnitCB(),
                            SanitizeValue(coi_val),                       
                            LookupBiotaSpeciesCB(get_maris_species),
                            LookupBiotaBodyPartCB(get_maris_bodypart),                          
                            LookupBiogroupCB(partial(get_biogroup_lut, species_lut_path())),
                            LookupSedimentCB(get_maris_sediments),
                            LookupUnitCB(),
                            LookupDetectionLimitCB(),
                            RemapDataProviderSampleIdCB(),
                            LookupFiltCB()
                            ])

print(tfm()['seawater']['FILT'].unique())

[0 2 1]


***

#### ~~Lookup : Area~~

&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;*NetCDF4 format variable: ``area``.*

&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;*Open Refine format variable: Area is not included*

TODO : Write callback for area. Will I use the marineregions.org API to complete lookup? 

In [610]:
'''#| export 
def get_area_lut():
    df = pd.read_excel(area_lut_path(), usecols=['displayName','areaId'])
    return df.set_index('displayName').to_dict()['areaId']
'''

"#| export \ndef get_area_lut():\n    df = pd.read_excel(area_lut_path(), usecols=['displayName','areaId'])\n    return df.set_index('displayName').to_dict()['areaId']\n"

***

### ~~Sample Notes~~

&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;*NetCDF4 format variable: ``sample_notes``.*

&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;*Open Refine format variable: ``Sample notes
``*

>  HELCOM data does not include ``sample_notes``. 

***

### ~~Measurement Notes~~

&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;*NetCDF4 format variable: ``measurement_notes``.*

&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;*Open Refine format variable: ``Measurement notes``*

>  HELCOM data does not include ``measurement_notes``. 

***

### Station ID 

&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;*NetCDF4 format variable: Station ID is not included.*

&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;*Open Refine format variable: ``Station ID``*

>  MARIS NetCDF4 format does not include Station ID.

In [611]:
# | export
class RemapStationIdCB(Callback):
    "Remap Station ID to MARIS format."

    def __init__(self):
        """
        Initialize the RemapStationIdCB with no specific parameters.
        """
        fc.store_attr()

    def __call__(self, tfm: 'Transformer'):
        """
        Iterate through all DataFrames in the transformer object and remap 'STATION' to 'station_id'.

        Args:
            tfm (Transformer): The transformer object containing DataFrames.
        """
        for grp in tfm.dfs.keys():
            self._remap_station_id(tfm.dfs[grp])

    def _remap_station_id(self, df: pd.DataFrame):
        """
        Remap 'STATION' column to 'station_id' in the given DataFrame.

        Args:
            df (pd.DataFrame): The DataFrame to modify.
        """
        df['station_id'] = df['STATION']

In [612]:
#|eval: false
dfs = load_data(fname_in)
tfm = Transformer(dfs, cbs=[LowerStripRdnNameCB(),
                            RemapRdnNameCB(),
                            ParseTimeCB(),
                            EncodeTimeCB(cfg()),                             
                            NormalizeUncUnitCB(),
                            SanitizeValue(coi_val),                       
                            LookupBiotaSpeciesCB(get_maris_species),
                            LookupBiotaBodyPartCB(get_maris_bodypart),                          
                            LookupBiogroupCB(partial(get_biogroup_lut, species_lut_path())),
                            LookupSedimentCB(get_maris_sediments),
                            LookupUnitCB(),
                            LookupDetectionLimitCB(),
                            RemapDataProviderSampleIdCB(),
                            RemapStationIdCB()
                            ])

print(tfm()['seawater']['station_id'].unique())

['RU10' 'RU11' 'RU19' 'RU20' 'RU23' 'RU25' 'RU32' 'RU52' 'RU89' 'RU94*'
 'RU99' 'RU141' 'RU156' 'B6' 'B50' 'BY15' 'BY28' 'L6' 'L7' 'L8' 'LА-11'
 'L13' 'Т1' 'RU5' 'B46' 'P16' 'Z1' 'SM1' 'SM2' 'SM5' 'P116' 'K11' 'P110'
 '2N2' 'ZN2' 'P14' 'P63' 'P140' 'P2' 'P3' 'P5' 'P39' 'B12' 'B13' 'B15'
 'K3' 'M3' 'R4' 'P1' 'P40' 'A' 'GDR250' 'Z' 'K6' 'P127' 'P104' 'NP' '4P7'
 'MW4' 'ZN4' 'EDC19' 'STOLGR' 'SCHLEI' 'KALKGR' 'KLBELT' 'KN' 'EDC20'
 'GBELT2' 'EDC38' 'EDC42' 'EDC32' 'EDC46' 'EDC47' 'KATT1' 'EDC55' 'SOUNDA'
 'SUND2' 'EDC58' 'ARKO1' 'BY2' 'EDC65' 'EDC68' 'GDR213' 'EDC80' 'EDC101'
 'EDC88' '2' 'EDC73' 'BHOLM3' 'EDC63' '18' 'DARSS' 'KOTN11' 'KFOTN6'
 'EDC35' 'EDC16' 'EDC7' 'EDC148' 'FBELT1' 'EDC92' '87/21' 'EDC70' 'EDC66'
 'GDR109' 'GDR069' 'EDC18' 'EDC17' 'EDC39' 'ZZZ' 'EDC93' 'EDC81' 'GBELT1'
 'EDC40' '8' 'EDC145' 'EDC36' '11' 'EDC54' 'EDC53' 'SUND4' 'SUND3' 'SUND1'
 'BHOLM2' 'EDC75' '22' '29' 'EDC821' 'EDC82' 'EDC89' 'EDC98' 'EDC119'
 'EDC114' '33' 'EDC96' 'EDC118' 'EDC109' '37' 'EDC100' '39

***

#TODO : Helcom SEQUENCE is 'SEQUENCE: SEQUENCE NUMBER CONSISTS OF YEAR AND SEQUENTAL SAMPLING NUMBER WHICH IS THE SAME AS IN THE KEY (7 LAST CHARACTERS OF THE KEY)   ' which is different to Profile ID or Transect ID


### Profile ID, Transect ID or Sequence ID

&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;*NetCDF4 format variable: Profile ID, Transect ID or Sequence ID is not included.*

&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;*Open Refine format variable: ``Profile or transect ID``*

>  MARIS NetCDF4 format does not include Profile ID, Transect ID or Sequence ID.

In [613]:
tfm.dfs['biota']['SEQUENCE']

0        2012041
1        2012041
2        2012041
3        2012041
4        2012040
          ...   
14888    2018021
14889    2018022
14890    2018022
14891    2018022
14892    2018001
Name: SEQUENCE, Length: 14873, dtype: int64

In [614]:
# | export
class RemapProfileIdCB(Callback):
    "Remap Profile ID to MARIS format."

    def __init__(self):
        """
        Initialize the RemapProfileIdCB with no specific parameters.
        """
        fc.store_attr()

    def __call__(self, tfm: 'Transformer'):
        """
        Iterate through all DataFrames in the transformer object and remap 'SEQUENCE' to 'profile_or_transect_id'.

        Args:
            tfm (Transformer): The transformer object containing DataFrames.
        """
        for grp in tfm.dfs.keys():
            self._remap_profile_id(tfm.dfs[grp])

    def _remap_profile_id(self, df: pd.DataFrame):
        """
        Remap 'SEQUENCE' column to 'profile_or_transect_id' in the given DataFrame.

        Args:
            df (pd.DataFrame): The DataFrame to modify.
        """
        df['profile_or_transect_id'] = df['SEQUENCE']


In [615]:
#|eval: false
dfs = load_data(fname_in)
tfm = Transformer(dfs, cbs=[                         
                            LowerStripRdnNameCB(),
                            RemapRdnNameCB(),
                            ParseTimeCB(),
                            EncodeTimeCB(cfg()),                             
                            NormalizeUncUnitCB(),
                            SanitizeValue(coi_val),                       
                            LookupBiotaSpeciesCB(get_maris_species),
                            LookupBiotaBodyPartCB(get_maris_bodypart),                          
                            LookupBiogroupCB(partial(get_biogroup_lut, species_lut_path())),
                            LookupSedimentCB(get_maris_sediments),
                            LookupUnitCB(),
                            LookupDetectionLimitCB(),                           
                            RemapDataProviderSampleIdCB(),
                            RemapStationIdCB(),
                            RemapProfileIdCB()
                            ])

print(tfm()['seawater']['profile_or_transect_id'].unique())


[2012003 2012004 2012005 ...  201806  201807  201808]


***

### Sediment slice position (top and bottom)

&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;*NetCDF4 format variable: Top and Bottom is not included.*

&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;*Open Refine format variables: ``Top`` and ``Bottom``.*

>  MARIS NetCDF4 format does not include sediment slice top and bottom.

In [616]:
# | export
class RemapSedSliceTopBottomCB(Callback):
    "Remap Sediment slice top and bottom to MARIS format."

    def __init__(self):
        """
        Initialize the RemapSedSliceTopBottomCB with no specific parameters.
        """
        fc.store_attr()

    def __call__(self, tfm: 'Transformer'):
        """
        Iterate through all DataFrames in the transformer object and remap sediment slice top and bottom.

        Args:
            tfm (Transformer): The transformer object containing DataFrames.
        """
        if 'sediment' in tfm.dfs:
            self._remap_sediment_slice(tfm.dfs['sediment'])

    def _remap_sediment_slice(self, df: pd.DataFrame):
        """
        Remap 'LOWSLI' column to 'bottom' and 'UPPSLI' column to 'top' in the given DataFrame.

        Args:
            df (pd.DataFrame): The DataFrame to modify.
        """
        df['bottom'] = df['LOWSLI']
        df['top'] = df['UPPSLI']


In [617]:
#|eval: false
dfs = load_data(fname_in)
tfm = Transformer(dfs, cbs=[LowerStripRdnNameCB(),
                            RemapRdnNameCB(),
                            ParseTimeCB(),
                            EncodeTimeCB(cfg()),                             
                            NormalizeUncUnitCB(),
                            SanitizeValue(coi_val),                       
                            LookupBiotaSpeciesCB(get_maris_species),
                            LookupBiotaBodyPartCB(get_maris_bodypart),                          
                            LookupBiogroupCB(partial(get_biogroup_lut, species_lut_path())),
                            LookupSedimentCB(get_maris_sediments),
                            LookupUnitCB(),
                            LookupDetectionLimitCB(),    
                            RemapDataProviderSampleIdCB(),
                            RemapStationIdCB(),
                            RemapProfileIdCB(), 
                            RemapSedSliceTopBottomCB()
                            ])

print(tfm()['sediment']['top'].head())


0    15.0
1    20.0
2     0.0
3     2.0
4     4.0
Name: top, dtype: float64


***

### Dry to wet ratio

&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;*NetCDF4 format variable: DW% is not included.*

&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;*Open Refine format variables: ``Dry/wet ratio``.*

HELCOM Description:

**Sediment:**
1. DW%: DRY WEIGHT AS PERCENTAGE (%) OF FRESH WEIGHT.
2. VALUE_Bq/kg: Measured radioactivity concentration in Bq/kg dry wt. in scientific format(e.g. 123 = 1.23E+02, 0.076 = 7.6E-02)

**Biota:**
1. WEIGHT: Average weight (in g) of specimen in the sample
2. DW%: DRY WEIGHT AS PERCENTAGE (%) OF FRESH WEIGHT

In [618]:
# | export
class LookupDryWetRatio(Callback):
    "Lookup dry-wet ratio and format for MARIS."

    def __init__(self):
        """
        Initialize the LookupDryWetRatio callback with no specific parameters.
        """
        fc.store_attr()

    def __call__(self, tfm: 'Transformer'):
        """
        Iterate through all DataFrames in the transformer object and apply the dry-wet ratio lookup.

        Args:
            tfm (Transformer): The transformer object containing DataFrames.
        """
        for grp in tfm.dfs.keys():
            if 'DW%' in tfm.dfs[grp].columns:
                self._apply_dry_wet_ratio(tfm.dfs[grp])

    def _apply_dry_wet_ratio(self, df: pd.DataFrame):
        """
        Apply dry-wet ratio conversion and formatting to the given DataFrame.

        Args:
            df (pd.DataFrame): The DataFrame to modify.
        """
        df['dry_wet_ratio'] = df['DW%']
        # Convert 'DW%' = 0% to NaN.
        df.loc[df['dry_wet_ratio'] == 0, 'dry_wet_ratio'] = np.NaN


In [619]:
#|eval: false
dfs = load_data(fname_in)
tfm = Transformer(dfs, cbs=[LowerStripRdnNameCB(),
                            RemapRdnNameCB(),
                            ParseTimeCB(),
                            EncodeTimeCB(cfg()),                             
                            NormalizeUncUnitCB(),
                            SanitizeValue(coi_val),                       
                            LookupBiotaSpeciesCB(get_maris_species),
                            LookupBiotaBodyPartCB(get_maris_bodypart),                          
                            LookupBiogroupCB(partial(get_biogroup_lut, species_lut_path())),
                            LookupSedimentCB(get_maris_sediments),
                            LookupUnitCB(),
                            LookupDetectionLimitCB(),    
                            RemapDataProviderSampleIdCB(),
                            RemapStationIdCB(),
                            RemapProfileIdCB(), 
                            RemapSedSliceTopBottomCB(),
                            LookupDryWetRatio(),
                            CompareDfsAndTfm(dfs)
                            ])

tfm()

print(pd.DataFrame.from_dict(tfm.compare_stats) , '\n')
                    

print(tfm.dfs['biota']['dry_wet_ratio'].head())


                                                    seawater  sediment  biota
Number of rows in dfs                                  20318     37347  14893
Number of rows in tfm.dfs                              20242     37089  14873
Number of dropped rows                                    76       258     20
Number of rows in tfm.dfs + Number of dropped rows     20318     37347  14893 

0    18.453
1    18.453
2    18.453
3    18.453
4    18.458
Name: dry_wet_ratio, dtype: float64


***

### Capture Coordinates

&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;*NetCDF4 format variables: ``lon``  and ``lat``*

&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;*Open Refine format variables: ``Longitude decimal`` and ``Latitude decimal``.*

Use decimal degree coordinates if available; otherwise, convert from degree-minute format to decimal degrees.

In [620]:
#| export
# Columns of interest coordinates
coi_coordinates = {
    'seawater': {
        'lon_d': 'LONGITUDE (dddddd)',
        'lat_d': 'LATITUDE (dddddd)',
        'lon_m': 'LONGITUDE (ddmmmm)',
        'lat_m': 'LATITUDE (ddmmmm)'
    },
    'biota': {
        'lon_d': 'LONGITUDE dddddd',
        'lat_d': 'LATITUDE dddddd',
        'lon_m': 'LONGITUDE ddmmmm',
        'lat_m': 'LATITUDE ddmmmm'
    },
    'sediment': {
        'lon_d': 'LONGITUDE (dddddd)',
        'lat_d': 'LATITUDE (dddddd)',
        'lon_m': 'LONGITUDE (ddmmmm)',
        'lat_m': 'LATITUDE (ddmmmm)'
    }
}

In [621]:
# | export
def ddmmmm2dddddd(ddmmmm):
    """
    Convert coordinates from 'ddmmmm' format to 'dddddd' format.
    
    Args:
        ddmmmm (float): Coordinates in 'ddmmmm' format where 'dd' are degrees and 'mmmm' are minutes.
    
    Returns:
        float: Coordinates in 'dddddd' format.
    """
    # Split into degrees and minutes
    mins, degs = modf(ddmmmm)
    # Convert minutes to decimal
    mins = mins * 100
    # Convert to 'dddddd' format
    return round(int(degs) + (mins / 60), 6)


In [622]:
# | export
class FormatCoordinates(Callback):
    """
    Format coordinates for MARIS. Converts coordinates from 'ddmmmm' to 'dddddd' format if needed.

    Args:
        coi (dict): Dictionary containing column names for longitude and latitude in various formats.
        fn_convert_cor (Callable): Function to convert coordinates from 'ddmmmm' to 'dddddd' format.
    """
    def __init__(self, coi: dict, fn_convert_cor: Callable[[float], float]):
        """
        Initialize the FormatCoordinates callback.

        Args:
            coi (dict): Column names mapping for coordinates.
            fn_convert_cor (Callable): Function to convert coordinates.
        """
        fc.store_attr()

    def __call__(self, tfm):
        """
        Apply formatting to coordinates in the DataFrame.

        Args:
            tfm (Transformer): The transformer object containing DataFrames.
        """
        for grp in tfm.dfs.keys():
            self._format_coordinates(tfm.dfs[grp], grp)

    def _format_coordinates(self, df: pd.DataFrame, grp: str):
        """
        Format coordinates in the DataFrame for a specific group.

        Args:
            df (pd.DataFrame): DataFrame to modify.
            grp (str): Group name to determine column names.
        """
        lon_col_d = self.coi[grp]['lon_d']
        lat_col_d = self.coi[grp]['lat_d']
        lon_col_m = self.coi[grp]['lon_m']
        lat_col_m = self.coi[grp]['lat_m']
        
        # Define condition where 'dddddd' format is not available or is zero
        condition = (
            (df[lon_col_d].isna() | (df[lon_col_d] == 0)) |
            (df[lat_col_d].isna() | (df[lat_col_d] == 0))
        )
        
        # Convert coordinates using the provided conversion function where needed
        df['lon'] = np.where(
            condition,
            df[lon_col_m].apply(self.fn_convert_cor),
            df[lon_col_d]
        )
        
        df['lat'] = np.where(
            condition,
            df[lat_col_m].apply(self.fn_convert_cor),
            df[lat_col_d]
        )


In [623]:
#|eval: false
dfs = load_data(fname_in)
tfm = Transformer(dfs, cbs=[LowerStripRdnNameCB(),
                            RemapRdnNameCB(),
                            ParseTimeCB(),
                            EncodeTimeCB(cfg()),                             
                            NormalizeUncUnitCB(),
                            SanitizeValue(coi_val),                       
                            LookupBiotaSpeciesCB(get_maris_species),
                            LookupBiotaBodyPartCB(get_maris_bodypart),                          
                            LookupBiogroupCB(partial(get_biogroup_lut, species_lut_path())),
                            LookupSedimentCB(get_maris_sediments),
                            LookupUnitCB(),
                            LookupDetectionLimitCB(),    
                            RemapDataProviderSampleIdCB(),
                            RemapStationIdCB(),
                            RemapProfileIdCB(), 
                            RemapSedSliceTopBottomCB(),
                            LookupDryWetRatio(),
                            FormatCoordinates(coi_coordinates, ddmmmm2dddddd),
                            CompareDfsAndTfm(dfs)
                            ])

tfm()

print(pd.DataFrame.from_dict(tfm.compare_stats) , '\n')
print(tfm.dfs['biota'][['lat','lon']])

                                                    seawater  sediment  biota
Number of rows in dfs                                  20318     37347  14893
Number of rows in tfm.dfs                              20242     37089  14873
Number of dropped rows                                    76       258     20
Number of rows in tfm.dfs + Number of dropped rows     20318     37347  14893 

             lat        lon
0      54.283333  12.316667
1      54.283333  12.316667
2      54.283333  12.316667
3      54.283333  12.316667
4      54.283333  12.316667
...          ...        ...
14888  54.583300  19.000000
14889  54.333300  15.500000
14890  54.333300  15.500000
14891  54.333300  15.500000
14892  54.363900  19.433300

[14873 rows x 2 columns]


***

### Sanitize coordinates

&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;*NetCDF4 format variables: ``lon``  and ``lat``*

&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;*Open Refine format variables: ``Longitude decimal`` and ``Latitude decimal``.*

Sanitize coordinates drops a row when both longitude & latitude equal 0 or data contains unrealistic longitude & latitude values. Converts longitude & latitude `,` separator to `.` separator."

In [624]:
#|eval: false
dfs = load_data(fname_in)
tfm = Transformer(dfs, cbs=[LowerStripRdnNameCB(),
                            RemapRdnNameCB(),
                            ParseTimeCB(),
                            EncodeTimeCB(cfg()),                             
                            NormalizeUncUnitCB(),
                            SanitizeValue(coi_val),                       
                            LookupBiotaSpeciesCB(get_maris_species),
                            LookupBiotaBodyPartCB(get_maris_bodypart),                          
                            LookupBiogroupCB(partial(get_biogroup_lut, species_lut_path())),
                            LookupSedimentCB(get_maris_sediments),
                            LookupUnitCB(),
                            LookupDetectionLimitCB(),    
                            RemapDataProviderSampleIdCB(),
                            RemapStationIdCB(),
                            RemapProfileIdCB(), 
                            RemapSedSliceTopBottomCB(),
                            LookupDryWetRatio(),
                            FormatCoordinates(coi_coordinates, ddmmmm2dddddd),
                            SanitizeLonLatCB(),
                            CompareDfsAndTfm(dfs)
                            ])

tfm()
print(pd.DataFrame.from_dict(tfm.compare_stats) , '\n')
print(tfm.dfs['biota'][['lat','lon']])


                                                    seawater  sediment  biota
Number of rows in dfs                                  20318     37347  14893
Number of rows in tfm.dfs                              20242     37089  14873
Number of dropped rows                                    76       258     20
Number of rows in tfm.dfs + Number of dropped rows     20318     37347  14893 

             lat        lon
0      54.283333  12.316667
1      54.283333  12.316667
2      54.283333  12.316667
3      54.283333  12.316667
4      54.283333  12.316667
...          ...        ...
14888  54.583300  19.000000
14889  54.333300  15.500000
14890  54.333300  15.500000
14891  54.333300  15.500000
14892  54.363900  19.433300

[14873 rows x 2 columns]


***

### Review DFS and TFM data

In [625]:
#|eval: false
dfs = load_data(fname_in)
tfm = Transformer(dfs, cbs=[LowerStripRdnNameCB(),
                            RemapRdnNameCB(),
                            ParseTimeCB(),
                            EncodeTimeCB(cfg()),                             
                            NormalizeUncUnitCB(),
                            SanitizeValue(coi_val),                       
                            LookupBiotaSpeciesCB(get_maris_species),
                            LookupBiotaBodyPartCB(get_maris_bodypart),                          
                            LookupBiogroupCB(partial(get_biogroup_lut, species_lut_path())),
                            LookupSedimentCB(get_maris_sediments),
                            LookupUnitCB(),
                            LookupDetectionLimitCB(),
                            RemapDataProviderSampleIdCB(),
                            RemapStationIdCB(),
                            RemapProfileIdCB(), 
                            RemapSedSliceTopBottomCB(),
                            LookupDryWetRatio(),
                            FormatCoordinates(coi_coordinates, ddmmmm2dddddd),
                            SanitizeLonLatCB(),
                            SanitizeValue(coi_val),
                            CompareDfsAndTfm(dfs)
                            ])

tfm()

print(pd.DataFrame.from_dict(tfm.compare_stats) , '\n')


                                                    seawater  sediment  biota
Number of rows in dfs                                  20318     37347  14893
Number of rows in tfm.dfs                              20242     37089  14873
Number of dropped rows                                    76       258     20
Number of rows in tfm.dfs + Number of dropped rows     20318     37347  14893 



In [626]:
seawater_review=tfm.dfs_dropped['seawater']
biota_review=tfm.dfs_dropped['biota']
sediment_review=tfm.dfs_dropped['sediment']

***

In [627]:
tfm.dfs['seawater'].columns

Index(['KEY', 'NUCLIDE', 'METHOD', '< VALUE_Bq/m³', 'VALUE_Bq/m³', 'ERROR%_m³',
       'DATE_OF_ENTRY_x', 'COUNTRY', 'LABORATORY', 'SEQUENCE', 'DATE', 'YEAR',
       'MONTH', 'DAY', 'STATION', 'LATITUDE (ddmmmm)', 'LATITUDE (dddddd)',
       'LONGITUDE (ddmmmm)', 'LONGITUDE (dddddd)', 'TDEPTH', 'SDEPTH', 'SALIN',
       'TTEMP', 'FILT', 'MORS_SUBBASIN', 'HELCOM_SUBBASIN', 'DATE_OF_ENTRY_y',
       'time', 'uncertainty', 'value', 'unit', 'detection_limit',
       'data_provider_sample_id', 'station_id', 'profile_or_transect_id',
       'lon', 'lat'],
      dtype='object')

### Rename columns of interest for NetCDF or Open Refine

> Column names are standardized to MARIS NetCDF format (i.e. PEP8 ). 

In [651]:
#| export
# Define columns of interest (keys) and renaming rules (values).
def get_renaming_rules(encoding_type='netcdf'):
    vars = cdl_cfg()['vars']
    
    if encoding_type == 'netcdf':
        return {
            ('seawater', 'biota', 'sediment'): {
                # DEFAULT
                'lat': vars['defaults']['lat']['name'],
                'lon': vars['defaults']['lon']['name'],
                'time': vars['defaults']['time']['name'],
                'NUCLIDE': 'nuclide',
                'detection_limit': vars['suffixes']['detection_limit']['name'],
                'unit': vars['suffixes']['unit']['name'],
                'value': 'value',
                'uncertainty': vars['suffixes']['uncertainty']['name'],
                'counting_method': vars['suffixes']['counting_method']['name'],
                'sampling_method': vars['suffixes']['sampling_method']['name'],
                'preparation_method': vars['suffixes']['preparation_method']['name']
            },
            ('seawater',): {
                # SEAWATER
                'SALIN': vars['suffixes']['salinity']['name'],
                'SDEPTH': vars['defaults']['smp_depth']['name'],
                #'FILT': vars['suffixes']['filtered']['name'], Need to fix
                'TTEMP': vars['suffixes']['temperature']['name'],
                'TDEPTH': vars['defaults']['tot_depth']['name'],

            },
            ('biota',): {
                # BIOTA
                'SDEPTH': vars['defaults']['smp_depth']['name'],
                'species': vars['bio']['species']['name'],
                'body_part': vars['bio']['body_part']['name'],
                'bio_group': vars['bio']['bio_group']['name']
            },
            ('sediment',): {
                # SEDIMENT
                'sed_type': vars['sed']['sed_type']['name'],
                'TDEPTH': vars['defaults']['tot_depth']['name'],
            }
        }
    
    elif encoding_type == 'openrefine':
        return {
            ('seawater', 'biota', 'sediment'): {
                # DEFAULT
                'latitude': 'lat',
                'longitude': 'lon',
                'begperiod': 'time',
                'endperiod': 'endperiod',
                'nuclide_id': 'NUCLIDE',
                'detection': 'detection_limit',
                'unit_id': 'unit',
                'activity': 'value',
                'uncertaint': 'uncertainty',
                'vartype': 'vartype',
                'rangelow': 'rangelow',
                'rangeupp': 'rangeupp',
                'rl_detection': 'rl_detection',
                'ru_detection': 'ru_detection',
                'freq': 'freq',
                'sampdepth': 'SDEPTH',
                'samparea': 'samparea',
                'salinity': 'SALIN',
                'temperatur': 'TTEMP',
                'filtered': 'FILT',
                'oxygen': 'oxygen',
                'sampquality': 'sampquality',
                'station': 'station',
                'samplabcode': 'samplabcode',
                'profile': 'profile',
                'transect': 'transect',
                'IODE_QualityFlag': 'IODE_QualityFlag',
                'totdepth': 'TDEPTH',
                'counting_method': 'counmet_id',
                'sampling_method': 'sampmet_id',
                'preparation_method': 'prepmet_id',
                'sampnote': 'sampnote',
                'measurenote': 'measurenote'
            },
            ('seawater',): {
                # SEAWATER
                'volume': 'volume',
                'filtpore': 'filtpore',
                'acid': 'acid'
            },
            ('biota',): {
                # BIOTA
                'TaxonRepName': 'TaxonRepName',
                'species_id': 'species',
                'bodypar_id': 'body_part',
                'drywt': 'drywt',
                'wetwt': 'wetwt',
                'percentwt': 'percentwt',
                'drymet_id': 'drymet_id'
            },
            ('sediment',): {
                # SEDIMENT
                'sedtype_id': 'sed_type',
                'sedtrap': 'sedtrap',
                'sliceup': 'sliceup',
                'slicedown': 'slicedown',
                'SedRepName': 'SedRepName',
                'drywt': 'drywt',
                'wetwt': 'wetwt',
                'percentwt': 'percentwt',
                'drymet_id': 'drymet_id'
            }
        }
    
    else:
        print("Invalid encoding_type provided. Please use 'netcdf' or 'openrefine'.")
        return None


In [652]:
tfm.dfs['sediment'].columns

Index(['time', 'lon', 'tot_depth', 'lat', 'sed_type', 'ac228_dl', 'ag110m_dl',
       'am241_dl', 'ba140_dl', 'be7_dl',
       ...
       'sb124', 'sb125', 'sr90', 'th228', 'th232', 'th234', 'tl208', 'u235',
       'zn65', 'zr95'],
      dtype='object', length=177)

In [653]:
#| export
class SelectAndRenameColumnCB(Callback):
    """
    A callback to select and rename columns in a DataFrame based on provided renaming rules
    for a specified encoding type. It also prints renaming rules that were not applied
    because their keys were not found in the DataFrame.
    """
    
    def __init__(self, fn_renaming_rules, encoding_type='netcdf', verbose=False):
        """
        Initialize the SelectAndRenameColumnCB callback.

        Args:
            fn_renaming_rules (function): A function that returns a dictionary of renaming rules.
            encoding_type (str): The encoding type ('netcdf' or 'openrefine') to determine which renaming rules to use.
            verbose (bool): Whether to print out renaming rules that were not applied.
        """
        fc.store_attr()

    def __call__(self, tfm):
        """
        Apply column selection and renaming to DataFrames in the transformer, and identify unused rules.

        Args:
            tfm (Transformer): The transformer object containing DataFrames.
        """
        try:
            renaming_rules = self.fn_renaming_rules(self.encoding_type)
        except ValueError as e:
            print(f"Error fetching renaming rules: {e}")
            return

        for group in tfm.dfs.keys():
            # Get relevant renaming rules for the current group
            group_rules = self._get_group_rules(renaming_rules, group)

            if not group_rules:
                continue

            # Apply renaming rules and track keys not found in the DataFrame
            df = tfm.dfs[group]
            df, not_found_keys = self._apply_renaming(df, group_rules)
            tfm.dfs[group] = df
            
            # Print any renaming rules that were not used
            if not_found_keys and self.verbose:
                print(f"\nGroup '{group}' has the following renaming rules not applied:")
                for old_col in not_found_keys:
                    print(f"Key '{old_col}' from renaming rules was not found in the DataFrame.")

    def _get_group_rules(self, renaming_rules, group):
        """
        Retrieve and merge renaming rules for the specified group based on the encoding type.

        Args:
            renaming_rules (dict): Dictionary of all renaming rules.
            group (str): Group name to filter rules.

        Returns:
            dict: A dictionary of renaming rules applicable to the specified group.
        """
        relevant_rules = [rules for key, rules in renaming_rules.items() if group in key]
        merged_rules = {}
        for rules in relevant_rules:
            merged_rules.update(rules)
        return merged_rules

    def _apply_renaming(self, df, rename_rules):
        """
        Select columns based on renaming rules and apply renaming, only for existing columns.

        Args:
            df (pd.DataFrame): DataFrame to modify.
            rename_rules (dict): Dictionary of column renaming rules.

        Returns:
            tuple: A tuple containing:
                - The DataFrame with columns renamed and filtered.
                - A set of column names from renaming rules that were not found in the DataFrame.
        """
        existing_columns = set(df.columns)
        valid_rules = {old_col: new_col for old_col, new_col in rename_rules.items() if old_col in existing_columns}

        # Ensure that columns to keep includes both existing and valid new columns
        columns_to_keep = set(valid_rules.keys()).union(valid_rules.values())
        columns_to_keep = columns_to_keep.intersection(existing_columns)
        df = df[list(columns_to_keep)]
        
        # Apply renaming
        df.rename(columns=valid_rules, inplace=True)

        # Determine which keys were not found
        not_found_keys = set(rename_rules.keys()) - existing_columns
        return df, not_found_keys


In [654]:

#|eval: false
dfs = load_data(fname_in)
tfm = Transformer(dfs, cbs=[LowerStripRdnNameCB(),
                            RemapRdnNameCB(),
                            ParseTimeCB(),
                            EncodeTimeCB(cfg()),                             
                            NormalizeUncUnitCB(),
                            SanitizeValue(coi_val),                       
                            LookupBiotaSpeciesCB(get_maris_species),
                            LookupBiotaBodyPartCB(get_maris_bodypart),                          
                            LookupBiogroupCB(partial(get_biogroup_lut, species_lut_path())),
                            LookupSedimentCB(get_maris_sediments),
                            LookupUnitCB(),
                            LookupDetectionLimitCB(),
                            RemapDataProviderSampleIdCB(),
                            RemapStationIdCB(),
                            RemapProfileIdCB(), 
                            RemapSedSliceTopBottomCB(),
                            LookupDryWetRatio(),
                            FormatCoordinates(coi_coordinates, ddmmmm2dddddd),
                            SanitizeLonLatCB(),
                            SelectAndRenameColumnCB(get_renaming_rules, encoding_type='netcdf', verbose = True),
                            CompareDfsAndTfm(dfs)
                            ])

tfm()

#print(pd.DataFrame.from_dict(tfm.compare_stats) , '\n')


Group 'seawater' has the following renaming rules not applied:
Key 'preparation_method' from renaming rules was not found in the DataFrame.
Key 'sampling_method' from renaming rules was not found in the DataFrame.
Key 'counting_method' from renaming rules was not found in the DataFrame.

Group 'sediment' has the following renaming rules not applied:
Key 'preparation_method' from renaming rules was not found in the DataFrame.
Key 'sampling_method' from renaming rules was not found in the DataFrame.
Key 'counting_method' from renaming rules was not found in the DataFrame.

Group 'biota' has the following renaming rules not applied:
Key 'preparation_method' from renaming rules was not found in the DataFrame.
Key 'sampling_method' from renaming rules was not found in the DataFrame.
Key 'counting_method' from renaming rules was not found in the DataFrame.


{'seawater':        smp_depth  _sal        time nuclide  tot_depth  _temp      lon  value  \
 0            0.0   NaN  1337731200   cs137        NaN    NaN  29.3333    5.3   
 1           29.0   NaN  1337731200   cs137        NaN    NaN  29.3333   19.9   
 2            0.0   NaN  1339891200   cs137        NaN    NaN  23.1500   25.5   
 3            0.0   NaN  1337817600   cs137        NaN    NaN  27.9833   17.0   
 4           39.0   NaN  1337817600   cs137        NaN    NaN  27.9833   22.2   
 ...          ...   ...         ...     ...        ...    ...      ...    ...   
 20313        4.0   7.5  1434931200    sr90       12.0    NaN  14.2000    6.6   
 20314        4.0   7.7  1435017600    sr90       20.0    NaN  14.6672    6.9   
 20315        4.0   7.8  1435017600    sr90       17.0    NaN  14.3342    6.8   
 20316        4.0   8.4  1435104000    sr90       47.0    NaN  13.5002    7.3   
 20317       45.0  15.9  1435104000    sr90       47.0    NaN  13.5002    5.5   
 
            la

***

### Reshape: long to wide

Convert data from long to wide and rename columns to comply with NetCDF format.

In [655]:

#| export
class ReshapeLongToWide(Callback):
    "Convert data from long to wide with renamed columns."
    def __init__(self, columns=['nuclide'], values=['value']):
        fc.store_attr()
        # Retrieve all possible derived vars (e.g 'unc', 'dl', ...) from configs
        self.derived_cols = [value['name'] for value in cdl_cfg()['vars']['suffixes'].values()]
    
    def renamed_cols(self, cols):
        "Flatten columns name"
        return [inner if outer == "value" else f'{inner}{outer}'
                if inner else outer
                for outer, inner in cols]

    def pivot(self, df):
        # Among all possible 'derived cols' select the ones present in df
        derived_coi = [col for col in self.derived_cols if col in df.columns]
        df.index.name = 'org_index'
        df=df.reset_index()
        idx = list(set(df.columns) - set(self.columns + derived_coi + self.values))
        
        # Create a fill_value to replace NaN values in the columns used as the index in the pivot table.
        # Check if num_fill_value is already in the dataframe index values. If num_fill_value already exists
        # then increase num_fill_value by 1 until a value is found for num_fill_value that is not in the dataframe. 
        num_fill_value = -999
        while (df[idx] == num_fill_value).any().any():
            num_fill_value += 1
        # Fill in nan values for each col found in idx. 
        for col in idx:   
            if pd.api.types.is_numeric_dtype(df[col]):
                fill_value = num_fill_value
            if pd.api.types.is_string_dtype(df[col]):
                fill_value = 'NOT AVAILABLE'
                
            df[col]=df[col].fillna(fill_value)

        pivot_df=df.pivot_table(index=idx,
                              columns=self.columns,
                              values=self.values + derived_coi,
                              fill_value=np.nan,
                              aggfunc=lambda x: x
                              ).reset_index()
        

        # Replace fill_value  with  np.nan
        pivot_df[idx]=pivot_df[idx].replace({'NOT AVAILABLE': np.nan,
                                             num_fill_value : np.nan})
        # Set the index to be the org_index
        pivot_df = pivot_df.set_index('org_index')
                
        return (pivot_df)

    def __call__(self, tfm):
        for grp in tfm.dfs.keys():
            tfm.dfs[grp] = self.pivot(tfm.dfs[grp])
            tfm.dfs[grp].columns = self.renamed_cols(tfm.dfs[grp].columns)
            
    

In [656]:
#|eval: false
dfs = load_data(fname_in)
tfm = Transformer(dfs, cbs=[LowerStripRdnNameCB(),
                            RemapRdnNameCB(),
                            ParseTimeCB(),
                            EncodeTimeCB(cfg()),                             
                            NormalizeUncUnitCB(),
                            SanitizeValue(coi_val),                       
                            LookupBiotaSpeciesCB(get_maris_species),
                            LookupBiotaBodyPartCB(get_maris_bodypart),                          
                            LookupBiogroupCB(partial(get_biogroup_lut, species_lut_path())),
                            LookupSedimentCB(get_maris_sediments),
                            LookupUnitCB(),
                            LookupDetectionLimitCB(),
                            RemapDataProviderSampleIdCB(),
                            RemapStationIdCB(),
                            RemapProfileIdCB(), 
                            RemapSedSliceTopBottomCB(),
                            LookupDryWetRatio(),
                            FormatCoordinates(coi_coordinates, ddmmmm2dddddd),
                            SanitizeLonLatCB(),
                            SelectAndRenameColumnCB(get_renaming_rules, encoding_type='netcdf'),
                            ReshapeLongToWide(), 
                            CompareDfsAndTfm(dfs)

                            ])
tfm()
print(tfm.dfs['biota'].head())
print(pd.DataFrame.from_dict(tfm.compare_stats) , '\n')

                time  smp_depth      lon  body_part  species      lat  \
org_index                                                               
7824       442540800        0.0  20.0000          7       50  56.0000   
7826       442540800        0.0  20.0000          7       99  56.0000   
7825       442540800        0.0  20.0000         52       50  56.0000   
7827       442540800        0.0  20.0000         52       99  56.0000   
5103       448070400        0.0  12.9083         54       96  55.7633   

           bio_group  ac228_dl  ag108m_dl  ag110m_dl  ...  sr89  sr90  tc99  \
org_index                                             ...                     
7824               4       NaN        NaN        NaN  ...   NaN   2.0   NaN   
7826               4       NaN        NaN        NaN  ...   NaN   3.4   NaN   
7825               4       NaN        NaN        NaN  ...   NaN   NaN   NaN   
7827               4       NaN        NaN        NaN  ...   NaN   NaN   NaN   
5103          

In [657]:
seawater_dfs_review=tfm.dfs['seawater']
biota_dfs_review=tfm.dfs['biota']
sediment_dfs_review=tfm.dfs['sediment']

***

## NetCDF encoder

### Example change logs

In [658]:
#|eval: false
dfs = load_data(fname_in)
tfm = Transformer(dfs, cbs=[LowerStripRdnNameCB(),
                            RemapRdnNameCB(),
                            ParseTimeCB(),
                            EncodeTimeCB(cfg()),                             
                            NormalizeUncUnitCB(),
                            SanitizeValue(coi_val),                       
                            LookupBiotaSpeciesCB(get_maris_species),
                            LookupBiotaBodyPartCB(get_maris_bodypart),                          
                            LookupBiogroupCB(partial(get_biogroup_lut, species_lut_path())),
                            LookupSedimentCB(get_maris_sediments),
                            LookupUnitCB(),
                            LookupDetectionLimitCB(),
                            RemapDataProviderSampleIdCB(),
                            RemapStationIdCB(),
                            RemapProfileIdCB(), 
                            RemapSedSliceTopBottomCB(),
                            LookupDryWetRatio(),
                            FormatCoordinates(coi_coordinates, ddmmmm2dddddd),
                            SanitizeLonLatCB(),
                            SelectAndRenameColumnCB(get_renaming_rules, encoding_type='netcdf'),
                            ReshapeLongToWide(), 
                            #CompareDfsAndTfm(dfs)
                            ])

# Transform
tfm()
# Check transformation logs
tfm.logs

['Convert nuclide names to lowercase and strip any trailing spaces.',
 'Remap radionuclide names to MARIS radionuclide names.',
 'Encode time as `int` representing seconds since xxx',
 'Convert from relative error % to uncertainty of activity unit.',
 "Sanitize value by removing blank entries and ensuring the 'value' column is retained.",
 '\n    Biota species remapped to MARIS db:\n        CARD EDU: Cerastoderma edule\n        LAMI SAC: Saccharina latissima\n        PSET MAX: Scophthalmus maximus\n        STIZ LUC: Sander luciopercas\n    ',
 "\n    Update bodypart id based on MARIS dbo_bodypar.xlsx:\n        - 3: 'Whole animal eviscerated without head',\n        - 12: 'Viscera',\n        - 8: 'Skin'\n    ",
 '\n    Update biogroup id based on MARIS dbo_species.xlsx\n    ',
 '\n    Update sediment id based on MARIS dbo_sedtype.xlsx.\n    ',
 'Remap value type to MARIS format.',
 "\n    Remap 'KEY' column to 'data_provider_sample_id' in each DataFrame.\n    ",
 'Remap Station ID to MAR

### Feed global attributes

In [659]:
#| export
kw = ['oceanography', 'Earth Science > Oceans > Ocean Chemistry> Radionuclides',
      'Earth Science > Human Dimensions > Environmental Impacts > Nuclear Radiation Exposure',
      'Earth Science > Oceans > Ocean Chemistry > Ocean Tracers, Earth Science > Oceans > Marine Sediments',
      'Earth Science > Oceans > Ocean Chemistry, Earth Science > Oceans > Sea Ice > Isotopes',
      'Earth Science > Oceans > Water Quality > Ocean Contaminants',
      'Earth Science > Biological Classification > Animals/Vertebrates > Fish',
      'Earth Science > Biosphere > Ecosystems > Marine Ecosystems',
      'Earth Science > Biological Classification > Animals/Invertebrates > Mollusks',
      'Earth Science > Biological Classification > Animals/Invertebrates > Arthropods > Crustaceans',
      'Earth Science > Biological Classification > Plants > Macroalgae (Seaweeds)']


In [660]:
#| export
def get_attrs(tfm, zotero_key, kw=kw):
    return GlobAttrsFeeder(tfm.dfs, cbs=[
        BboxCB(),
        DepthRangeCB(),
        TimeRangeCB(cfg()),
        ZoteroCB(zotero_key, cfg=cfg()),
        KeyValuePairCB('keywords', ', '.join(kw)),
        KeyValuePairCB('publisher_postprocess_logs', ', '.join(tfm.logs))
        ])()

In [661]:
#|eval: false
get_attrs(tfm, zotero_key=zotero_key, kw=kw)

{'geospatial_lat_min': '31.17',
 'geospatial_lat_max': '65.75',
 'geospatial_lon_min': '9.6333',
 'geospatial_lon_max': '53.5',
 'geospatial_bounds': 'POLYGON ((9.6333 53.5, 31.17 53.5, 31.17 65.75, 9.6333 65.75, 9.6333 53.5))',
 'time_coverage_start': '1984-01-10T00:00:00',
 'time_coverage_end': '2018-12-14T00:00:00',
 'title': 'Environmental database - Helsinki Commission Monitoring of Radioactive Substances',
 'summary': 'MORS Environment database has been used to collate data resulting from monitoring of environmental radioactivity in the Baltic Sea based on HELCOM Recommendation 26/3.\n\nThe database is structured according to HELCOM Guidelines on Monitoring of Radioactive Substances (https://www.helcom.fi/wp-content/uploads/2019/08/Guidelines-for-Monitoring-of-Radioactive-Substances.pdf), which specifies reporting format, database structure, data types and obligatory parameters used for reporting data under Recommendation 26/3.\n\nThe database is updated and quality assured annua

In [662]:
#| export
def enums_xtra(tfm, vars):
    "Retrieve a subset of the lengthy enum as 'species_t' for instance"
    enums = Enums(lut_src_dir=lut_path(), cdl_enums=cdl_cfg()['enums'])
    xtras = {}
    for var in vars:
        unique_vals = tfm.unique(var)
        if unique_vals.any():
            xtras[f'{var}_t'] = enums.filter(f'{var}_t', unique_vals)
    return xtras

### Encoding NETCDF

In [663]:
#| export
def encode(fname_in, fname_out_nc, nc_tpl_path, **kwargs):
    dfs = load_data(fname_in)
    tfm = Transformer(dfs, cbs=[LowerStripRdnNameCB(),
                                RemapRdnNameCB(),
                                ParseTimeCB(),
                                EncodeTimeCB(cfg()),                             
                                NormalizeUncUnitCB(),
                                SanitizeValue(coi_val),                       
                                LookupBiotaSpeciesCB(get_maris_species),
                                LookupBiotaBodyPartCB(get_maris_bodypart),                          
                                LookupBiogroupCB(partial(get_biogroup_lut, species_lut_path())),
                                LookupSedimentCB(get_maris_sediments),
                                LookupUnitCB(),
                                LookupDetectionLimitCB(),
                                RemapDataProviderSampleIdCB(),
                                RemapStationIdCB(),
                                RemapProfileIdCB(), 
                                RemapSedSliceTopBottomCB(),
                                LookupDryWetRatio(),
                                FormatCoordinates(coi_coordinates, ddmmmm2dddddd),
                                SanitizeLonLatCB(),
                                SelectAndRenameColumnCB(get_renaming_rules, encoding_type='netcdf'),
                                ReshapeLongToWide()
                                ])
    tfm()
    encoder = NetCDFEncoder(tfm.dfs, 
                            src_fname=nc_tpl_path,
                            dest_fname=fname_out_nc, 
                            global_attrs=get_attrs(tfm, zotero_key=zotero_key, kw=kw),
                            verbose=kwargs.get('verbose', False),
                            enums_xtra=enums_xtra(tfm, vars=['species', 'body_part'])
                           )
    encoder.encode()

In [664]:
#|eval: false
encode(fname_in, fname_out_nc, nc_tpl_path(), verbose=True)

--------------------------------------------------------------------------------
Group: seawater, Variable: lon
--------------------------------------------------------------------------------
Group: seawater, Variable: lat
--------------------------------------------------------------------------------
Group: seawater, Variable: smp_depth
--------------------------------------------------------------------------------
Group: seawater, Variable: tot_depth
--------------------------------------------------------------------------------
Group: seawater, Variable: time
--------------------------------------------------------------------------------
Group: seawater, Variable: h3
--------------------------------------------------------------------------------
Group: seawater, Variable: h3_unc
--------------------------------------------------------------------------------
Group: seawater, Variable: h3_dl
--------------------------------------------------------------------------------
Group:

***

## TODO

TODO: Include FILT for NetCDF

TODO : Do we want to include laboratory code in NetCDF?

TODO: Check sediment 'DW%' data that is less than 1%. Is this realistic? Check the 'DW%' data that is 0%. Run below before SelectAndRenameColumnCB. 

In [ ]:
dfs = load_data(fname_in)
tfm = Transformer(dfs, cbs=[LowerStripRdnNameCB(),
                            RemapRdnNameCB(),
                            ParseTimeCB(),
                            EncodeTimeCB(cfg()),                             
                            NormalizeUncUnitCB(),
                            LookupBiotaSpeciesCB(get_maris_species),
                            LookupBiotaBodyPartCB(get_maris_bodypart), 
                            LookupBiogroupCB(partial(get_biogroup_lut, species_lut_path())),
                            LookupSedimentCB(get_maris_sediments),
                            LookupUnitCB(),
                            LookupDetectionLimitCB(),
                            RemapDataProviderSampleIdCB(),
                            RemapStationIdCB(),
                            RemapProfileIdCB(), 
                            RemapSedSliceTopBottomCB(),
                            LookupDryWetRatio(),
                            FormatCoordinates(coi_coordinates, ddmmmm2dddddd),
                            SanitizeLonLatCB(),
                            SanitizeValue(coi_val)
                            ])
tfm()

{'seawater':                 KEY NUCLIDE  METHOD < VALUE_Bq/m³  VALUE_Bq/m³  ERROR%_m³  \
 0      WKRIL2012003   cs137     NaN           NaN          5.3     1.6960   
 1      WKRIL2012004   cs137     NaN           NaN         19.9     3.9800   
 2      WKRIL2012005   cs137     NaN           NaN         25.5     5.1000   
 3      WKRIL2012006   cs137     NaN           NaN         17.0     4.9300   
 4      WKRIL2012007   cs137     NaN           NaN         22.2     3.9960   
 ...             ...     ...     ...           ...          ...        ...   
 20313  WDHIG2015227    sr90  DHIG02           NaN          6.6     0.4950   
 20314  WDHIG2015237    sr90  DHIG02           NaN          6.9     0.5175   
 20315  WDHIG2015239    sr90  DHIG02           NaN          6.8     0.5100   
 20316  WDHIG2015255    sr90  DHIG02           NaN          7.3     0.5475   
 20317  WDHIG2015256    sr90  DHIG02           NaN          5.5     0.4180   
 
          DATE_OF_ENTRY_x  COUNTRY LABORATORY  SEQ

In [ ]:
grp='sediment'
check_data_sediment=tfm.dfs[grp][(tfm.dfs[grp]['DW%'] < 1) & (tfm.dfs[grp]['DW%'] > 0.001) ]
#check_data_sediment

In [ ]:
grp='sediment'
check_data_sediment=tfm.dfs[grp][(tfm.dfs[grp]['DW%'] == 0) ]
check_data_sediment

,KEY,NUCLIDE,METHOD,< VALUE_Bq/kg,VALUE_Bq/kg,ERROR%_kg,< VALUE_Bq/m²,VALUE_Bq/m²,ERROR%_m²,DATE_OF_ENTRY_x,...,unit,detection_limit,data_provider_sample_id,station_id,profile_or_transect_id,bottom,top,dry_wet_ratio,lon,lat
9824,SERPC1997001,cs134,NaN,NaN,3.80,0.7600,NaN,5.75,NaN,NaN,...,4,1,SERPC1997001,EE17,1997001.0,2.0,0.0,NaN,25.0167,59.7167
9825,SERPC1997001,cs137,NaN,NaN,389.00,15.5600,NaN,589.00,NaN,NaN,...,4,1,SERPC1997001,EE17,1997001.0,2.0,0.0,NaN,25.0167,59.7167
9826,SERPC1997002,cs134,NaN,NaN,4.78,0.6214,NaN,12.00,NaN,NaN,...,4,1,SERPC1997002,EE17,1997002.0,4.0,2.0,NaN,25.0167,59.7167
9827,SERPC1997002,cs137,NaN,NaN,420.00,16.8000,NaN,1060.00,NaN,NaN,...,4,1,SERPC1997002,EE17,1997002.0,4.0,2.0,NaN,25.0167,59.7167
9828,SERPC1997003,cs134,NaN,NaN,3.12,0.5304,NaN,12.00,NaN,NaN,...,4,1,SERPC1997003,EE17,1997003.0,6.0,4.0,NaN,25.0167,59.7167
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
15257,SKRIL1999062,th228,1,NaN,68.00,NaN,NaN,NaN,NaN,NaN,...,4,0,SKRIL1999062,RU23,1999062.0,15.0,10.0,NaN,25.5000,59.8167
15258,SKRIL1999063,k40,1,NaN,1210.00,NaN,NaN,NaN,NaN,NaN,...,4,0,SKRIL1999063,RU23,1999063.0,21.5,15.0,NaN,25.5000,59.8167
15259,SKRIL1999063,ra226,KRIL01,NaN,56.50,NaN,NaN,NaN,NaN,NaN,...,4,0,SKRIL1999063,RU23,1999063.0,21.5,15.0,NaN,25.5000,59.8167
15260,SKRIL1999063,ra228,KRIL01,NaN,72.20,NaN,NaN,NaN,NaN,NaN,...,4,0,SKRIL1999063,RU23,1999063.0,21.5,15.0,NaN,25.5000,59.8167


In [ ]:
grp='biota'
check_data_sediment=tfm.dfs[grp][(tfm.dfs[grp]['DW%'] == 0) ]
check_data_sediment

,KEY,NUCLIDE,METHOD,< VALUE_Bq/kg,VALUE_Bq/kg,BASIS,ERROR%,NUMBER,DATE_OF_ENTRY_x,COUNTRY,...,body_part,bio_group,unit,detection_limit,data_provider_sample_id,station_id,profile_or_transect_id,dry_wet_ratio,lon,lat
5971,BERPC1997002,k40,NaN,NaN,116.00,W,3.4800,NaN,NaN,91.0,...,52,4,5,1,BERPC1997002,Paldiski,1997002,NaN,24.1667,59.3667
5972,BERPC1997002,cs137,NaN,NaN,12.60,W,0.5040,NaN,NaN,91.0,...,52,4,5,1,BERPC1997002,Paldiski,1997002,NaN,24.1667,59.3667
5973,BERPC1997002,cs134,NaN,NaN,0.14,W,0.0252,NaN,NaN,91.0,...,52,4,5,1,BERPC1997002,Paldiski,1997002,NaN,24.1667,59.3667
5974,BERPC1997001,k40,NaN,NaN,116.00,W,4.6400,NaN,NaN,91.0,...,52,4,5,1,BERPC1997001,Paldiski,1997001,NaN,24.1667,59.3667
5975,BERPC1997001,cs137,NaN,NaN,12.00,W,0.4800,NaN,NaN,91.0,...,52,4,5,1,BERPC1997001,Paldiski,1997001,NaN,24.1667,59.3667
5976,BERPC1997001,cs134,NaN,NaN,0.21,W,0.0504,NaN,NaN,91.0,...,52,4,5,1,BERPC1997001,Paldiski,1997001,NaN,24.1667,59.3667


TODO: Note weight definition in HELCOM is different than in MARIS.  HELCOM definition is 'Average weight (in g) of specimen in the sample. MARIS definition is 'dry weight of biota sample in grammes (g).' or 'wet weight of biota sample in grammes (g).'.

TODO :  For biota we have some entries that include a YEAR but no month and day (see tfm.dfs_dropped['biota'][['YEAR', 'MONTH', 'DAY', 'DATE']]). For these entries I have included a day and month as 1`. 

TODO: Review the format of 'ddmmmm'.. Example of data '29.2000'. Assuming that this is 29 degrees. Then 20 minutes with a remainder of .00 minutes. 

TODO: Follow up with HELCOM on entries where the value is nan. 

TODO: Review Maris Open Refine date format. The description format is 'DD-MMM-YYYY'. The example format is '29/Sep/2008'.  